In [190]:
# -*- coding: utf-8 -*-
"""
Created on Thurs June 27 2024 8:29am

@author: BMZ7571
"""

import simpy
from simpy.util import start_delayed
import random

import pandas as pd
import numpy as np

In [191]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Parameters

##### Lead Times

In [192]:
# Lead Times

#US Plants
PLANTS_TO_RDC = random.randint(5,7) # Time it takes for pallets to move from US Plants to the RDC's in days
PLANTS_TO_DSD = random.randint(5,7) # Time it takes for pallets to move from US Plants to the DSD Branches in days

#SALINAS
SALINAS_TO_RDC = random.randint(10,14) # Time it takes for pallets to move from salinas to the RDC's in days
SALINAS_TO_DSD = random.randint(10,14) # Time it takes for pallets to move from salinas to DSD Branches in days
SALINAS_TO_CUSTOMERS = random.randint(10, 14) # Time it takes for pallets to move from salinas directly to customers in days


#RDC
RDC_TO_CUSTOMER = random.randint(2,4) # Time it takes for pallets to move from RDC's to customers in days
RDC_TO_DSD = random.randint(4,5) # Time it takes for pallets to move from RDC's to DSD Branches in days

#DSD
DSD_TO_CUSTOMER = 2 # Time it takes for pallets to move from DSD Branches to and from customers in days

#REPACK
REPACK_TO_DSD = random.randint(5,7) # Time it takes pallets to move from repack to dsd branches in days
REPACK_TO_CUSTOMERS = random.randint(5,7) # Time it takes pallets to move from repack to customers in days

#RETURN
END_TO_SALINAS = random.randint(14,21) # Time it takes to move pallets back to Salinas in days
END_TO_PLANT = random.randint(15,15) # Time it takes to move pallets back to Plants in days
END_TO_REPACK = random.randint(7,10) # Time it takes to move pallets back to Repack in days
END_TO_EM = random.randint(10,14) # Tim it takes to move pallets back to EMs in days

##### Holding Times

In [193]:
#Holding Times

#US PLANTS
PLANTS_HOLDING_TIME = random.randint(7,8) # Time pallets sit before they can be moved out of US Plants in days

#SALINAS
SALINAS_HOLDING_TIME = random.randint(7,10) # Time pallets sit before they are moved out of Salinas in days

#RDC
RDC_HOLDING_TIME = random.randint(14,28) # Time pallets sit before they are moved out of RDC's in days

#DSD
DSD_HOLDING_TIME = random.randint(7,10) # Time pallets sit before they are moved out of DSD Branches in days

#REPACK
REPACK_HOLDING_TIME = random.randint(5,7) # Time pallets sit before they are moved out of Repack in days

##### Distribution Percentages

In [194]:
#RDC Pallet Distribution Percentages
RDC_TO_DSD_DIST = 0.54
RDC_TO_CUSTOMER_DIST = 0.38
RDC_TO_REPACK_DIST = 0.08

#### Return Costs

In [195]:
#Mexico
RETURN_TO_MEXICO_COST = 6000

#US Plants
RETURN_TO_PLANTS_COST = 2100

#EMs
RETURN_TO_EM_COST = 2100

#Repack
RETURN_TO_REPACK_COST = 2100

##### Loss and Damage Rates

In [196]:
# Loss and Damage

#LOSS RATE
PALLET_LOSS_RATE = 0.06

#DAMAGE RATE BY TURN
PALLET_DAMAGE_RATE_T1 = 0.04
PALLET_DAMAGE_RATE_T2 = 0.08
PALLET_DAMAGE_RATE_T3 = 0.16
PALLET_DAMAGE_RATE_T4 = 0.32
PALLET_DAMAGE_RATE_T5 = 0.64

##### Pallet Costs

In [197]:
#Pallet Cost

#Cost per stringer pallet
STRINGER_COST = 14

#Cost per block pallet in US
BLOCK_US_COST = 22

#Cost per block pallet in Mexico
BLOCK_MEXICO_COST = 19

#### Truck Capacity

In [198]:
#Amount of WW block pallets that can fit on one truck
truck_capacity = .54

------

#### Initializer Variables

In [199]:
#Initialize pallets to return bucket for each turn in the recycling process
pallets_available_to_return_t2 = 0
pallets_available_to_return_t3 = 0
pallets_available_to_return_t4 = 0
pallets_available_to_return_t5 = 0


#Initialize pallet loss bucket
pallets_lost_to_customers = 0

#Initialize output list to collect weekly data
output = []

#Validation
val = []

#Location based return
return_by_location =[]

-----

# Define Locations

#### Salinas

In [200]:
class Mexico_Hub(object):
    def __init__(self, env, initial_pallets):
        #Link Salinas to Simpy environment
        self.env = env
        #Set starting pallet level at location
        self.initial_pallets = initial_pallets

        #Set capacity resource for processes to use
        self.capacity = simpy.Resource(env, capacity = 1000)

        #Initialize storage containers for each turn
        self.pallet_reserve_t1 = simpy.Container(env, init = initial_pallets)
        self.pallet_reserve_t2 = simpy.Container(env, init = 0)
        self.pallet_reserve_t3 = simpy.Container(env, init = 0)
        self.pallet_reserve_t4 = simpy.Container(env, init = 0)
        self.pallet_reserve_t5 = simpy.Container(env, init = 0)

        #Create container to track pallets in process of return at any given time step
        self.pallet_reserve_intransit = simpy.Container(env, init = initial_pallets)

        #Create container to track pallet need at location throughout simulation
        self.pallet_need = simpy.Container(env, init = 0)

        #Run pallet monitoring process defined below
        self.mon_proc = env.process(self.monitor_pallet_level(env))

    def monitor_pallet_level(self, env):
        while True:
            val.append({'time': env.now, 'Salinas T1 Pallet Level' : self.pallet_reserve_t1.level, 'Salinas T2 Pallet Level' : self.pallet_reserve_t2.level, 'Salinas T3 Pallet Level' : self.pallet_reserve_t3.level, 
            'Salinas T4 Pallet Level' : self.pallet_reserve_t4.level, 'Salinas T5 Pallet Level' : self.pallet_reserve_t5.level, 'Salinas Pallet Level + In Transit' : self.pallet_reserve_intransit.level, 'Salinas Hub Pallet Need': self.pallet_need.level})

            #Run this process at every timestep
            yield env.timeout(1)

#### US Bakeries

In [201]:
class Richmond(object):
    def __init__(self, env, initial_pallets):
        #link Richmond to Simpy environment
        self.env = env
        #Set starting pallet level at location
        self.initial_pallets = initial_pallets

        #Set capacity resource for processes to use
        self.capacity = simpy.Resource(env, capacity= 1000)

        #Initialize storage containers for each turn
        self.pallet_reserve_t1 = simpy.Container(env, init = initial_pallets)
        self.pallet_reserve_t2 = simpy.Container(env, init = 0)
        self.pallet_reserve_t3 = simpy.Container(env, init = 0)
        self.pallet_reserve_t4 = simpy.Container(env, init = 0)
        self.pallet_reserve_t5 = simpy.Container(env, init = 0)

        #Create container to track pallets in process of return at any given time step
        self.pallet_reserve_intransit = simpy.Container(env, init = initial_pallets)

        #Create container to track pallet need at location throughout simulation
        self.pallet_need = simpy.Container(env, init = 0)

        #Run pallet monitoring process defined below
        self.mon_proc = env.process(self.sim_output(env))
    
    def sim_output(self, env):
        '''This process runs at every time step to take a look each conatiner for this location and append relevant info to the output'''
        global pallets_available_to_return_t2
        global pallets_available_to_return_t3
        global pallets_available_to_return_t4
        global pallets_available_to_return_t5
        global weekly_return_counter
        global output
        while True:
            if len(str(env.now//7)) == 1:
                week = '0' + str((env.now//7) + 1)
            else:
                week = str((env.now//7) + 1)
            #Check to see if current time step is divisible by 7 for idiosynracy in simulation flow that does not print correct output for %7 timesteps (only relevant for specific locations due to unpredictability of threading)
            #if env.now % 7 == 0:
            output.append({'week': week, 'day': env.now, 't2': pallets_available_to_return_t2, 't3': pallets_available_to_return_t3,
                            'expected_pallet_surplus': sum([pallets_available_to_return_t2, pallets_available_to_return_t3, pallets_available_to_return_t4, pallets_available_to_return_t5])})
    
            #Run this process at every timestep
            yield env.timeout(1)

In [202]:
class Portland(object):
    def __init__(self, env, initial_pallets):
        #Link Portland to Simpy environment
        self.env = env
        #Set starting pallet level at location
        self.initial_pallets = initial_pallets

        #Set capacity resource for processes to use
        self.capacity = simpy.Resource(env, capacity = 1000)

        #Initialize storage containers for each turn
        self.pallet_reserve_t1 = simpy.Container(env, init = initial_pallets)
        self.pallet_reserve_t2 = simpy.Container(env, init = 0)
        self.pallet_reserve_t3 = simpy.Container(env, init = 0)
        self.pallet_reserve_t4 = simpy.Container(env, init = 0)
        self.pallet_reserve_t5 = simpy.Container(env, init = 0)

        #Create container to track pallets in process of return at any given time step
        self.pallet_reserve_intransit = simpy.Container(env, init = initial_pallets)

        #Create container to track pallet need at location throughout simulation
        self.pallet_need = simpy.Container(env, init = 0)

In [203]:
class Chicago(object):
    def __init__(self, env, initial_pallets):
        #Link Chicago to Simpy environment
        self.env = env
        #Set starting pallet level at location
        self.initial_pallets = initial_pallets

        #Set capacity resource for processes to use
        self.capacity = simpy.Resource(env, capacity = 1000)

        #Initialize storage containers for each turn
        self.pallet_reserve_t1 = simpy.Container(env, init = initial_pallets)
        self.pallet_reserve_t2 = simpy.Container(env, init = 0)
        self.pallet_reserve_t3 = simpy.Container(env, init = 0)
        self.pallet_reserve_t4 = simpy.Container(env, init = 0)
        self.pallet_reserve_t5 = simpy.Container(env, init = 0)

        #Create container to track pallets in process of return at any given time step
        self.pallet_reserve_intransit = simpy.Container(env, init = initial_pallets)

        #Create container to track pallet need at location throughout simulation
        self.pallet_need = simpy.Container(env, init = 0)


In [204]:
class Naperville(object):
    def __init__(self, env, initial_pallets):
        #Link Naperville to Simpy environment
        self.env = env
        #Set starting pallet level at location
        self.initial_pallets = initial_pallets

        #Set capacity resource for processes to use
        self.capacity = simpy.Resource(env, capacity = 1000)

        #Initialize storage containers for each turn
        self.pallet_reserve_t1 = simpy.Container(env, init = initial_pallets)
        self.pallet_reserve_t2 = simpy.Container(env, init = 0)
        self.pallet_reserve_t3 = simpy.Container(env, init = 0)
        self.pallet_reserve_t4 = simpy.Container(env, init = 0)
        self.pallet_reserve_t5 = simpy.Container(env, init = 0)

        #Create container to track pallets in process of return at any given time step
        self.pallet_reserve_intransit = simpy.Container(env, init = initial_pallets)

        #Create container to track pallet need at location throughout simulation
        self.pallet_need = simpy.Container(env, init = 0)

------

### Functions

In [205]:
def pallet_logic(pallets):
    '''Checks number of pallets sent out and applies MDLZ WW Block logic to them'''

    if pallets > 30:
        pallets = 30
    
    return pallets

# Data Ingestion

In [206]:
#Location Mapping
location_map_df = pd.read_excel(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\Simulation\Location Map.xlsx")

#Drop plants and random locations without good data
location_map_df = location_map_df.dropna()

#Convert IBL to integer for mapping
location_map_df['IBL'] = location_map_df['IBL'].astype(int)

#Create location mapping dictionary for each location
location_map = dict(zip(location_map_df['IBL'], location_map_df['Plant Type']))

In [207]:
#Operational Week Mapping
operational_week_map = pd.read_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Orlando Cart Test\Data\operational_week_map.csv")

#Convert date format to datetime
operational_week_map['Date'] = pd.to_datetime(operational_week_map['Date'])

In [208]:
#Salinas Outbound Data
salinas_outbound = pd.read_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\SAP Data\Salinas_Outbound_Pallets.csv")

#Convert date format to datetime
salinas_outbound['Delivery Date'] = pd.to_datetime(salinas_outbound['Delivery Date'], format = '%Y%m%d')

#Filter data to 2024 and append operational week
salinas_outbound = salinas_outbound.merge(operational_week_map, left_on = 'Delivery Date', right_on = 'Date').drop('Date', axis =1 )

#Map IBLs to location types for demand sorting
salinas_outbound['Dest Plant Type'] = salinas_outbound['DEST_KEY'].map(location_map)

#Calculate outbound pallet logic
salinas_outbound['Picked Pallets'] = salinas_outbound['Picked Pallets'].apply(lambda x: pallet_logic(x))

In [209]:
#Richmond Outbound Data
richmond_outbound = pd.read_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\SAP Data\Richmond_Outbound_Pallets.csv")
richmond_1144_outbound = pd.read_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\SAP Data\Richmond_1144_Outbound_Pallets.csv")

#Combine Richmond outbound files before grouping
richmond_outbound = pd.concat([richmond_outbound, richmond_1144_outbound])

#Convert date format to datetime
richmond_outbound['Delivery Date'] = pd.to_datetime(richmond_outbound['Delivery Date'], format = '%Y%m%d')

#Filter data to 2024 and append operational week
richmond_outbound = richmond_outbound.merge(operational_week_map, left_on = 'Delivery Date', right_on = 'Date').drop('Date', axis =1 )

#Map IBLs to location types for demand sorting
richmond_outbound['Dest Plant Type'] = richmond_outbound['DEST_KEY'].map(location_map)

#Calculate outbound pallet logic
richmond_outbound['Picked Pallets'] = richmond_outbound['Picked Pallets'].apply(lambda x: pallet_logic(x))

In [210]:
#Portland Outbound Data
portland_outbound = pd.read_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\SAP Data\Portland_1143_Outbound_Pallets.csv")

#Convert date format to datetime
portland_outbound['Delivery Date'] = pd.to_datetime(portland_outbound['Delivery Date'], format = '%Y%m%d')

#Filter data to 2024 and append operational week
portland_outbound = portland_outbound.merge(operational_week_map, left_on = 'Delivery Date', right_on = 'Date').drop('Date', axis =1 )

#Map IBLs to location types for demand sorting
portland_outbound['Dest Plant Type'] = portland_outbound['DEST_KEY'].map(location_map)

#Calculate outbound pallet logic
portland_outbound['Picked Pallets'] = portland_outbound['Picked Pallets'].apply(lambda x: pallet_logic(x))

In [211]:
#Chicago Outbound Data
chicago_outbound = pd.read_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\SAP Data\Chicago_Outbound_Pallets.csv")

#Convert date format to datetime
chicago_outbound['Delivery Date'] = pd.to_datetime(chicago_outbound['Delivery Date'], format = '%Y%m%d')

#Filter data to 2024 and append operational week
chicago_outbound = chicago_outbound.merge(operational_week_map, left_on = 'Delivery Date', right_on = 'Date').drop('Date', axis =1 )

#Map IBLs to location types for demand sorting
chicago_outbound['Dest Plant Type'] = chicago_outbound['DEST_KEY'].map(location_map)

#Calculate outbound pallet logic
chicago_outbound['Picked Pallets'] = chicago_outbound['Picked Pallets'].apply(lambda x: pallet_logic(x))

In [212]:
#Naperville Outbound Data
naperville_outbound = pd.read_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\SAP Data\Naperville_Outbound_Pallets.csv")

#Convert date format to datetime
naperville_outbound['Delivery Date'] = pd.to_datetime(naperville_outbound['Delivery Date'], format = '%Y%m%d')

#Filter data to 2024 and append operational week
naperville_outbound = naperville_outbound.merge(operational_week_map, left_on = 'Delivery Date', right_on = 'Date').drop('Date', axis =1)

#Map IBLs to location types for demand sorting
naperville_outbound['Dest Plant Type'] = naperville_outbound['DEST_KEY'].map(location_map)

#Calculate outbound pallet logic
#naperville_outbound['Picked Pallets'] = naperville_outbound['Picked Pallets'].apply(lambda x: pallet_logic(x))

# Processes

In [213]:
#Create return counter to track anticipated weekly returns for entire network
weekly_return_counter = [0] * 100

#### Salinas

In [214]:
#Group Salinas outbound data by branch type to use for lookups in each outbound process function
salinas_outbound_grouped = salinas_outbound.groupby(['Operational Week', 'Dest Plant Type'])['Picked Pallets'].sum().reset_index()

Salinas to DSD

In [215]:
#Input Salinas actual outbound to DSDs in list to index in processes
mexico_hub_dsd_weekly_outbound = list(salinas_outbound_grouped[salinas_outbound_grouped['Dest Plant Type'] == 'Branch']['Picked Pallets'].values)

In [216]:
#Extrapolate demand out a year
mexico_hub_dsd_weekly_outbound = mexico_hub_dsd_weekly_outbound + mexico_hub_dsd_weekly_outbound

In [217]:
def salinas_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Salinas plant to dsd node'''

    print('time', env.now, 'salinas_to_dsd running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Salinas to DSD
    pallets_sent = mexico_hub_dsd_weekly_outbound[week - 1]

    print('time', env.now, 'pallets_sent', pallets_sent)
    
    #Grab current pallet reserve container levels for Salinas
    curr_t1_level = mexico_hub.pallet_reserve_t1.level
    curr_t2_level = mexico_hub.pallet_reserve_t2.level
    curr_t3_level = mexico_hub.pallet_reserve_t3.level
    curr_t4_level = mexico_hub.pallet_reserve_t4.level
    curr_t5_level = mexico_hub.pallet_reserve_t5.level

    #Check if Salinas pallet reserve T1 pallets can cover weekly demand
    if mexico_hub.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Salinas Hub
        with mexico_hub.capacity.request() as req:
            yield req

            #Take out weekly pallet demand from T1 pallet supply
            yield mexico_hub.pallet_reserve_t1.get(pallets_sent)

            #Check if there is pallet need, if so set back to 0
            if mexico_hub.pallet_need.level > 0:
                yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)

            #Get following weeks outbound demand to use as pallet need
            yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])


            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Salinas to DSD
            yield env.timeout(SALINAS_TO_DSD)

        #Run next step in the model, sending pallets from Salinas to DSD Branches
        yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Second check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level >= pallets_sent:
        #Check if Salinas pallet reserve T2 pallets can cover weekly demand
        if mexico_hub.pallet_reserve_t2.level >= pallets_sent:
            
            #If true, only T2 pallets will be used
            pallets_sent_t1 = 0

            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(mexico_hub.pallet_reserve_t2.level - pallets_sent) < 0.01:
                pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
            else:
                pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)

                #Put pallet need into pallet need container
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)

            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use both T1 + T2 pallet supply to cover demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)


                yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)

                #Put pallet need into pallet need container
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)

            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Third check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Salinas pallet reserve T3 pallets can cover weekly demand
        if curr_t3_level >= pallets_sent:

            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)

                #Put pallet need into pallet need container
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)

            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2 & T3 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - mexico_hub.pallet_reserve_t2.level <= 0:
                #If true, no T1 pallets will be sent
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t2.level - mexico_hub.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = mexico_hub.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - mexico_hub.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and mexico_hub.pallet_reserve_t2.level > 0:
                    yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and mexico_hub.pallet_reserve_t3.level > 0:
                    yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)
                
                #Put pallet need into pallet need container
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fourth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level + mexico_hub.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Salinas pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield mexico_hub.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)

                #Put pallet need into pallet need conatiner
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)

            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - mexico_hub.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T2, T3 & T4 pallets to cover weekly demand
            elif pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - mexico_hub.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level - mexico_hub.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = mexico_hub.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and mexico_hub.pallet_reserve_t2.level > 0:
                    yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and mexico_hub.pallet_reserve_t3.level > 0:
                    yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                if mexico_hub.pallet_reserve_t4.level > 0:
                    yield mexico_hub.pallet_reserve_t4.get(pallets_sent_t4)

                    #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)

                #Put pallet need into pallet need container
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fifth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level + mexico_hub.pallet_reserve_t4.level + mexico_hub.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Salinas pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield mexico_hub.pallet_reserve_t5.get(pallets_sent_t5)

                #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)

                #Put pallet need into pallet need container
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)

            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - mexico_hub.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level < 0.01):
                    pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = curr_t5_level
            
            #Must use mix of T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - mexico_hub.pallet_reserve_t4.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = mexico_hub.pallet_reserve_t5.level

            #Must use mix of T2, T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = mexico_hub.pallet_reserve_t5.level
            
            #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level - mexico_hub.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = mexico_hub.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = mexico_hub.pallet_reserve_t5.level
        
            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and mexico_hub.pallet_reserve_t2.level > 0:
                    yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and mexico_hub.pallet_reserve_t3.level > 0:
                    yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and mexico_hub.pallet_reserve_t4.level > 0:
                    yield mexico_hub.pallet_reserve_t4.get(pallets_sent_t4)

                yield mexico_hub.pallet_reserve_t5.get(pallets_sent_t5)

                #Check if there is pallet need, if so set back to 0
                if mexico_hub.pallet_need.level > 0:
                    yield mexico_hub.pallet_need.get(mexico_hub.pallet_need.level)


                #Put pallet need into pallet need container
                yield mexico_hub.pallet_need.put(mexico_hub_dsd_weekly_outbound[week + 2] + mexico_hub_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Salinas to DSD
                yield env.timeout(SALINAS_TO_DSD)

            #Run next step in the model, sending pallets from the Salinas to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Salinas can fulfill with current total pallet level
        fillable_demand = mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level + mexico_hub.pallet_reserve_t4.level + mexico_hub.pallet_reserve_t5.level

        #Request Simpy Resource from Salinas
        with mexico_hub.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield mexico_hub.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield mexico_hub.pallet_need.get(pallets_sent - fillable_demand)
        
        #Re-run this process, sending pallets from Salinas to DSD
        yield env.process(salinas_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Salinas to RDC

In [218]:
#Input Salinas actual outbound to RDCs in list to index in processes
mexico_hub_rdc_weekly_outbound = mexico_hub_rdc_weekly_outbound = list(salinas_outbound_grouped[salinas_outbound_grouped['Dest Plant Type'] == 'RDC']['Picked Pallets'].values)

In [219]:
#Extrapolate demand out a year
mexico_hub_rdc_weekly_outbound = mexico_hub_rdc_weekly_outbound + mexico_hub_rdc_weekly_outbound

In [220]:
def salinas_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Salinas plant to RDC node'''

    print('time', env.now, 'salinas_to_rdc running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Salinas to DSD
    pallets_sent = mexico_hub_rdc_weekly_outbound[week - 1]

    print('time', env.now, 'pallets_sent', pallets_sent)

    #Grab current pallet reserve container levels for Salinas
    curr_t1_level = mexico_hub.pallet_reserve_t1.level
    curr_t2_level = mexico_hub.pallet_reserve_t2.level
    curr_t3_level = mexico_hub.pallet_reserve_t3.level
    curr_t4_level = mexico_hub.pallet_reserve_t4.level
    curr_t5_level = mexico_hub.pallet_reserve_t5.level

    #Check if Salinas pallet reserve T1 pallets can cover weekly demand
    if mexico_hub.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Salinas
        with mexico_hub.capacity.request() as req:
            yield req

            #Take pallets out of respective turn conatiner
            yield mexico_hub.pallet_reserve_t1.get(pallets_sent)
            #Put pallet need into pallet need container
            #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])


            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Richmon to RDCs
            yield env.timeout(SALINAS_TO_RDC)

        #Run next step in the model, sending pallets from the Salinas to RDCs
        yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level >= pallets_sent:
        #Check if Salinas pallet reserve T2 pallets can cover weekly demand
        if curr_t2_level >= pallets_sent:
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = 0
            pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                #Put pallet need into pallet need container
                #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)

            #Run next step in the model, sending pallets from the Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)

                yield mexico_hub.pallet_reserve_t2.get(mexico_hub.pallet_reserve_t2.level)

                #Put pallet need into pallet need conatiner
                #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)

            #Run next step in the model, sending pallets from the Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Salinas pallet reserve T3 pallets can cover weekly demand
        if mexico_hub.pallet_reserve_t3.level >= pallets_sent:
            
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(mexico_hub.pallet_reserve_t3.level - pallets_sent) < 0.0001:
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
            else:
                pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)

            #Run next step in the model, sending pallets from the Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - mexico_hub.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0
            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t2.level - mexico_hub.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = mexico_hub.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - mexico_hub.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and mexico_hub.pallet_reserve_t2.level > 0:
                    yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and mexico_hub.pallet_reserve_t3.level > 0:
                    yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                ##yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)
            
            #Run next step in the model, sending pallets from the Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level + mexico_hub.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Salinas pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield mexico_hub.pallet_reserve_t4.get(pallets_sent_t4)

                #Put pallet need into pallet need container
                #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)

            #Run next step in the model, sending pallets from Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - mexico_hub.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 pallets
            elif pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - mexico_hub.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level - mexico_hub.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = mexico_hub.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and mexico_hub.pallet_reserve_t2.level > 0:
                    yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and mexico_hub.pallet_reserve_t3.level > 0:
                    yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                yield mexico_hub.pallet_reserve_t4.get(pallets_sent_t4)

                #Put pallet need into pallet need container
                #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)
            
            #Run next step in the model, sending pallets from Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level + mexico_hub.pallet_reserve_t4.level + mexico_hub.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Salinas pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:

            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield mexico_hub.pallet_reserve_t5.get(pallets_sent_t5)

                #Put pallet need into pallet need container
                #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)

            #Run next step in the model, sending pallets from Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - mexico_hub.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - mexico_hub.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level) < 0.01:
                    pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                pallets_sent_t5 = curr_t5_level
            
            #Check to see if weekly demand can be covered only using T3 + T4 + T5 pallets
            elif pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - mexico_hub.pallet_reserve_t4.level 
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = mexico_hub.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 + T5 pallets
            elif pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = mexico_hub.pallet_reserve_t5.level
            
            #Must use a mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level - mexico_hub.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = mexico_hub.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - mexico_hub.pallet_reserve_t4.level - mexico_hub.pallet_reserve_t3.level - mexico_hub.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = mexico_hub.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = mexico_hub.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = mexico_hub.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = mexico_hub.pallet_reserve_t5.level

            #Request Simpy Resource from Salinas
            with mexico_hub.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and mexico_hub.pallet_reserve_t1.level > 0:
                    yield mexico_hub.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and mexico_hub.pallet_reserve_t2.level > 0:
                    yield mexico_hub.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and mexico_hub.pallet_reserve_t3.level > 0:
                    yield mexico_hub.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and mexico_hub.pallet_reserve_t4.level > 0:
                    yield mexico_hub.pallet_reserve_t4.get(pallets_sent_t4)

                yield mexico_hub.pallet_reserve_t5.get(pallets_sent_t5)


                #Put pallet need into pallet need container
                #yield mexico_hub.pallet_need.put(mexico_hub_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Salinas to RDCs
                yield env.timeout(SALINAS_TO_RDC)

            #Run next step in the model, sending pallets from Salinas to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Salinas can fulfill with current total pallet level
        fillable_demand = mexico_hub.pallet_reserve_t1.level + mexico_hub.pallet_reserve_t2.level + mexico_hub.pallet_reserve_t3.level + mexico_hub.pallet_reserve_t4.level + mexico_hub.pallet_reserve_t5.level
 
        #Request Simpy Resource from Salinas
        with mexico_hub.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield mexico_hub.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield mexico_hub.pallet_need.get(pallets_sent - fillable_demand)

        
        #Re-run this process, sending pallets from Salinas to RDC
        yield env.process(salinas_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Salinas Return

In [221]:
def dsd_return_salinas(env, mexico_hub):
    '''This process returns pallets back to Salinas'''

    #Locally initiate pallet availability containers
    global pallets_available_to_return_t2
    global pallets_available_to_return_t3
    global pallets_available_to_return_t4
    global pallets_available_to_return_t5

    #Indicate destination for model output
    destination = 'Salinas'

    #Get current avaialable to return pallet level for each turn container
    curr_pallets_t2 = pallets_available_to_return_t2
    curr_pallets_t3 = pallets_available_to_return_t3
    curr_pallets_t4 = pallets_available_to_return_t4
    curr_pallets_t5 = pallets_available_to_return_t5
    

    #Assign current Salinas Hub pallet need for use in formulas
    mexico_hub_need = mexico_hub.pallet_need.level
    print('LOOK', mexico_hub_need)

    #Check if Salinas has any pallet need in current time step
    if mexico_hub_need > 0:

        #Starting in reverse order, check if there is enough turn 5 availability to cover Salinas pallet need
        if curr_pallets_t5 >= mexico_hub_need:
            #If so, take Salinas need pallets out of available to return container
            pallets_available_to_return_t5 -= mexico_hub_need

            #Request Simpy Resource from Salinas
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Salinas
                yield mexico_hub.pallet_reserve_intransit.put(mexico_hub_need)

            #Pallets in transit from DSD back to Salinas
            yield env.timeout(END_TO_SALINAS)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield mexico_hub.pallet_reserve_intransit.get(mexico_hub_need)
                
                #Put current Salinas Hub pallet need back into Salinas Hub pallet reserve container once returned
                yield mexico_hub.pallet_reserve_t5.put(mexico_hub_need)

                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += mexico_hub_need


                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': mexico_hub_need})

                #Take pallet need out of Salinas Hub once returned
                #yield mexico_hub.pallet_need.get(mexico_hub_need)


        #Check if T4 + T5 pallets can cover Salinas need
        elif curr_pallets_t5 + curr_pallets_t4 >= mexico_hub_need:
            #All (if any) T5 pallets will be returned
            mexico_hub_need_t5 = curr_pallets_t5
            #Calculate T4 pallets that will be sent
            mexico_hub_need_t4 = mexico_hub_need - curr_pallets_t5

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 -= mexico_hub_need_t4


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req
                #Flag pallets are on their way back to Salinas
                yield mexico_hub.pallet_reserve_intransit.put(mexico_hub_need)

            #Pallets in transit from DSD back to Salinas
            yield env.timeout(END_TO_SALINAS)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield mexico_hub.pallet_reserve_intransit.get(mexico_hub_need)

                #Put current Salinas Hub pallet need back into Salinas Hub pallet reserve container once returned
                if mexico_hub_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Salinas T5 container
                    yield mexico_hub.pallet_reserve_t5.put(mexico_hub_need_t5)

                if mexico_hub_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Salinas T4 container
                    yield mexico_hub.pallet_reserve_t4.put(mexico_hub_need_t4)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += mexico_hub_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': mexico_hub_need})
                
                #Take pallet need out of Salinas container once pallets are returned
                #yield mexico_hub.pallet_need.get(mexico_hub_need)


        #Check if T3 + T4 + T5 pallets can cover Salinas need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 >= mexico_hub_need:
            #All (if any) T5 pallets will be returned
            mexico_hub_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            mexico_hub_need_t4 = curr_pallets_t4
            #Calculate T3 pallets that will be sent
            mexico_hub_need_t3 = mexico_hub_need - curr_pallets_t5 - curr_pallets_t4

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 -= mexico_hub_need_t3


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Salinas
                yield mexico_hub.pallet_reserve_intransit.put(mexico_hub_need)

            #Pallets in transit from DSD back to Salinas
            yield env.timeout(END_TO_SALINAS)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield mexico_hub.pallet_reserve_intransit.get(mexico_hub_need)

                #Put current Salinas Hub pallet need back into Salinas Hub pallet reserve container once returned
                if mexico_hub_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Salinas T5 container
                    yield mexico_hub.pallet_reserve_t5.put(mexico_hub_need_t5)

                if mexico_hub_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Salinas T4 container
                    yield mexico_hub.pallet_reserve_t4.put(mexico_hub_need_t4)

                if mexico_hub_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Salinas T3 container
                    yield mexico_hub.pallet_reserve_t3.put(mexico_hub_need_t3)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += mexico_hub_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': mexico_hub_need})

                #Take pallet need out of Salinas container once pallets are returned
                #yield mexico_hub.pallet_need.get(mexico_hub_need)


        #Check if T2 + T3 + T4 + T5 pallets can cover Salinas need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 >= mexico_hub_need:
            #All (if any) T5 pallets will be returned
            mexico_hub_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            mexico_hub_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            mexico_hub_need_t3 = curr_pallets_t3
            #Calculate T2 pallets that will be sent
            mexico_hub_need_t2 = mexico_hub_need - curr_pallets_t5 - curr_pallets_t4 - curr_pallets_t3
            print('TIME', env.now, 'pallets t2 BEFORE', pallets_available_to_return_t2)
            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t2 -= mexico_hub_need_t2

            print('TIME Salinas', env.now, 'need', mexico_hub_need)
            print('TIME', env.now, 't2 need', mexico_hub_need_t2)
            print('TIME', env.now, 't3 need', mexico_hub_need_t3)
            print('TIME', env.now, 'pallets t2', pallets_available_to_return_t2)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Salinas
                yield mexico_hub.pallet_reserve_intransit.put(mexico_hub_need)

            #Pallets in transit from DSD back to Salinas Hub
            yield env.timeout(END_TO_SALINAS)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield mexico_hub.pallet_reserve_intransit.get(mexico_hub_need)

                #Put current Salinas Hub pallet need back into Salinas Hub pallet reserve container once returned
                if mexico_hub_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Salinas T5 container
                    yield mexico_hub.pallet_reserve_t5.put(mexico_hub_need_t5)

                if mexico_hub_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Salinas T4 container
                    yield mexico_hub.pallet_reserve_t4.put(mexico_hub_need_t4)

                if mexico_hub_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Salinas T3 container
                    yield mexico_hub.pallet_reserve_t3.put(mexico_hub_need_t3)

                if mexico_hub_need_t2 > 0:
                    #Put pallets returned from T2 PATR container back into Salinas T2 container
                    yield mexico_hub.pallet_reserve_t2.put(mexico_hub_need_t2)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += mexico_hub_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': mexico_hub_need})
                
                #Take pallet need out of Salinas container once pallets are returned
                #yield mexico_hub.pallet_need.get(mexico_hub_need)


        #Can't fill full pallet need in Salinas with pallet return program, send everything that is available
        else:
            #All (if any) T5 pallets will be returned
            mexico_hub_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            mexico_hub_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            mexico_hub_need_t3 = curr_pallets_t3
            #All (if any) T2 pallets will be returned
            mexico_hub_need_t2 = curr_pallets_t2

            #Adjust pallets available to return containers
            pallets_available_to_return_t2 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t5 = 0

            #Check to make sure there are any pallets available to return
            if curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 > 0:
                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Flag pallets are on their way back to Salinas
                    yield mexico_hub.pallet_reserve_intransit.put(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                #Pallets in transit from DSD back to Salinas Hub
                yield env.timeout(END_TO_SALINAS)

                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Take out pallets in transit as they have arrived at the facility
                    yield mexico_hub.pallet_reserve_intransit.get(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                    #Put current Salinas Hub pallet need back into Salinas Hub pallet reserve container once returned
                    if mexico_hub_need_t5 > 0:
                        #Put pallets returned from T5 PATR container back into Salinas T5 container
                        yield mexico_hub.pallet_reserve_t5.put(mexico_hub_need_t5)

                    if mexico_hub_need_t4 > 0:
                        #Put pallets returned from T4 PATR container back into Salinas T4 container
                        yield mexico_hub.pallet_reserve_t4.put(mexico_hub_need_t4)

                    if mexico_hub_need_t3 > 0:
                        #Put pallets returned from T3 PATR container back into Salinas T3 container
                        yield mexico_hub.pallet_reserve_t3.put(mexico_hub_need_t3)

                    if mexico_hub_need_t2 > 0:
                        #Put pallets returned from T2 PATR container back into Salinas T2 container
                        yield mexico_hub.pallet_reserve_t2.put(mexico_hub_need_t2)
                    
                    print('Look here!')
                    print('mexico need t2:', mexico_hub_need_t2)
                    print('mexico need t3:', mexico_hub_need_t3)
                    
                    #Floor the simulation day to calculate current week's return
                    week = env.now//7
                    weekly_return_counter[week] += (mexico_hub_need_t2 + mexico_hub_need_t3 + mexico_hub_need_t4 + mexico_hub_need_t5)

                    #Add weekly return by location for dashboard use
                    return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': (mexico_hub_need_t2 + mexico_hub_need_t3 + mexico_hub_need_t4 + mexico_hub_need_t5)})

                    #Take pallet need out of Salinas container once pallets are returned
                    #yield mexico_hub.pallet_need.get(mexico_hub_need_t2 + mexico_hub_need_t3 + mexico_hub_need_t4 + mexico_hub_need_t5)


            #No pallets avaialable left to return in the current time step
            else:
                print('No pallets available to return in Salinas')

-----

#### Richmond

In [222]:
#Group Richmond outbound data by branch type to use for lookups in each outbound process function
richmond_outbound_grouped = richmond_outbound.groupby(['Operational Week', 'Dest Plant Type'])['Picked Pallets'].sum().reset_index()

Richmond to DSD

In [223]:
#Input Richmond actual outbound to DSDs in list to index in processes
richmond_dsd_weekly_outbound = list(richmond_outbound_grouped[richmond_outbound_grouped['Dest Plant Type'] == 'Branch']['Picked Pallets'].values)

#Extrapolate demand out a year
richmond_dsd_weekly_outbound = richmond_dsd_weekly_outbound + richmond_dsd_weekly_outbound

In [224]:
def richmond_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Richmond plant to dsd node'''

    print('time', env.now, 'richmond_to_dsd running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
        print('week', week)
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Richmond to DSD
    pallets_sent = richmond_dsd_weekly_outbound[week - 1]

    #Grab current pallet reserve container levels for Richmond
    curr_t1_level = richmond.pallet_reserve_t1.level
    curr_t2_level = richmond.pallet_reserve_t2.level
    curr_t3_level = richmond.pallet_reserve_t3.level
    curr_t4_level = richmond.pallet_reserve_t4.level
    curr_t5_level = richmond.pallet_reserve_t5.level

    #Check if Richmond pallet reserve T1 pallets can cover weekly demand
    if richmond.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Richmond Hub
        with richmond.capacity.request() as req:
            yield req

            #Take out weekly pallet demand from T1 pallet supply
            yield richmond.pallet_reserve_t1.get(pallets_sent)

            #Check if there is pallet need, if so set back to 0
            if richmond.pallet_need.level > 0:
                yield richmond.pallet_need.get(richmond.pallet_need.level)

            #Get following weeks outbound demand to use as pallet need
            yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])


            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Richmond to DSD
            yield env.timeout(PLANTS_TO_DSD)

        #Run next step in the model, sending pallets from Richmond to DSD Branches
        yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Second check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level >= pallets_sent:
        #Check if Richmond pallet reserve T2 pallets can cover weekly demand
        if richmond.pallet_reserve_t2.level >= pallets_sent:
            
            #If true, only T2 pallets will be used
            pallets_sent_t1 = 0

            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(richmond.pallet_reserve_t2.level - pallets_sent) < 0.01:
                pallets_sent_t2 = richmond.pallet_reserve_t2.level
            else:
                pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need container
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use both T1 + T2 pallet supply to cover demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)


                yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need container
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Third check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Richmond pallet reserve T3 pallets can cover weekly demand
        if curr_t3_level >= pallets_sent:

            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need container
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2 & T3 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - richmond.pallet_reserve_t2.level <= 0:
                #If true, no T1 pallets will be sent
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = richmond.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t2.level - richmond.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = richmond.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - richmond.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = richmond.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and richmond.pallet_reserve_t2.level > 0:
                    yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and richmond.pallet_reserve_t3.level > 0:
                    yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need container
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fourth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level + richmond.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Richmond pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield richmond.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need conatiner
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - richmond.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = richmond.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T2, T3 & T4 pallets to cover weekly demand
            elif pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = richmond.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - richmond.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level - richmond.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = richmond.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = richmond.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and richmond.pallet_reserve_t2.level > 0:
                    yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and richmond.pallet_reserve_t3.level > 0:
                    yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                if richmond.pallet_reserve_t4.level > 0:
                    yield richmond.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need container
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fifth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level + richmond.pallet_reserve_t4.level + richmond.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Richmond pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield richmond.pallet_reserve_t5.get(pallets_sent_t5)

                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need container
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - richmond.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level < 0.01):
                    pallets_sent_t4 = richmond.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = curr_t5_level
            
            #Must use mix of T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = richmond.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - richmond.pallet_reserve_t4.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = richmond.pallet_reserve_t5.level

            #Must use mix of T2, T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = richmond.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = richmond.pallet_reserve_t5.level
            
            #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level - richmond.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = richmond.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = richmond.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = richmond.pallet_reserve_t5.level
        
            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and richmond.pallet_reserve_t2.level > 0:
                    yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and richmond.pallet_reserve_t3.level > 0:
                    yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and richmond.pallet_reserve_t4.level > 0:
                    yield richmond.pallet_reserve_t4.get(pallets_sent_t4)

                yield richmond.pallet_reserve_t5.get(pallets_sent_t5)


                #Check if there is pallet need, if so set back to 0
                if richmond.pallet_need.level > 0:
                    yield richmond.pallet_need.get(richmond.pallet_need.level)

                #Put pallet need into pallet need container
                yield richmond.pallet_need.put(richmond_dsd_weekly_outbound[week + 2] + richmond_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Richmond to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Richmond to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Richmond can fulfill with current total pallet level
        fillable_demand = richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level + richmond.pallet_reserve_t4.level + richmond.pallet_reserve_t5.level

        #Request Simpy Resource from Richmond
        with richmond.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield richmond.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield richmond.pallet_need.get(pallets_sent - fillable_demand)
        
        #Re-run this process, sending pallets from Richmond to DSD
        yield env.process(richmond_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Richmond to RDC

In [225]:
#Input Richmond actual outbound to RDCs in list to index in processes
richmond_rdc_weekly_outbound = list(richmond_outbound_grouped[richmond_outbound_grouped['Dest Plant Type'] == 'RDC']['Picked Pallets'].values)

#Extrapolate demand out a year
richmond_rdc_weekly_outbound = richmond_rdc_weekly_outbound + richmond_rdc_weekly_outbound

In [226]:
def richmond_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Richmond plant to RDC node'''

    print('time', env.now, 'richmond_to_rdc running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Richmond to DSD
    pallets_sent = richmond_rdc_weekly_outbound[week - 1]

    #Grab current pallet reserve container levels for Richmond
    curr_t1_level = richmond.pallet_reserve_t1.level
    curr_t2_level = richmond.pallet_reserve_t2.level
    curr_t3_level = richmond.pallet_reserve_t3.level
    curr_t4_level = richmond.pallet_reserve_t4.level
    curr_t5_level = richmond.pallet_reserve_t5.level

    #Check if Richmond pallet reserve T1 pallets can cover weekly demand
    if richmond.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Richmond
        with richmond.capacity.request() as req:
            yield req

            #Take pallets out of respective turn conatiner
            yield richmond.pallet_reserve_t1.get(pallets_sent)

            #Put pallet need into pallet need container
            #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])


            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Richmon to RDCs
            yield env.timeout(PLANTS_TO_RDC)

        #Run next step in the model, sending pallets from the Richmond to RDCs
        yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level >= pallets_sent:
        #Check if Richmond pallet reserve T2 pallets can cover weekly demand
        if curr_t2_level >= pallets_sent:
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = 0
            pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                #Put pallet need into pallet need container
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)

                yield richmond.pallet_reserve_t2.get(richmond.pallet_reserve_t2.level)

                #Put pallet need into pallet need conatiner
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Richmond pallet reserve T3 pallets can cover weekly demand
        if richmond.pallet_reserve_t3.level >= pallets_sent:
            
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(richmond.pallet_reserve_t3.level - pallets_sent) < 0.0001:
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
            else:
                pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - richmond.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = richmond.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0
            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t2.level - richmond.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = richmond.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - richmond.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = richmond.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and richmond.pallet_reserve_t2.level > 0:
                    yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and richmond.pallet_reserve_t3.level > 0:
                    yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from the Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level + richmond.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Richmond pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield richmond.pallet_reserve_t4.get(pallets_sent_t4)
                    
                #Put pallet need into pallet need container
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - richmond.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = richmond.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 pallets
            elif pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = richmond.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - richmond.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level - richmond.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = richmond.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = richmond.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and richmond.pallet_reserve_t2.level > 0:
                    yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and richmond.pallet_reserve_t3.level > 0:
                    yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                yield richmond.pallet_reserve_t4.get(pallets_sent_t4)

                #Put pallet need into pallet need container
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level + richmond.pallet_reserve_t4.level + richmond.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Richmond pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:

            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield richmond.pallet_reserve_t5.get(pallets_sent_t5)

                #Put pallet need into pallet need container
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - richmond.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - richmond.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level) < 0.01:
                    pallets_sent_t4 = richmond.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                pallets_sent_t5 = curr_t5_level
            
            #Check to see if weekly demand can be covered only using T3 + T4 + T5 pallets
            elif pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = richmond.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - richmond.pallet_reserve_t4.level 
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = richmond.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 + T5 pallets
            elif pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = richmond.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = richmond.pallet_reserve_t5.level
            
            #Must use a mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level - richmond.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = richmond.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - richmond.pallet_reserve_t4.level - richmond.pallet_reserve_t3.level - richmond.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = richmond.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = richmond.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = richmond.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = richmond.pallet_reserve_t5.level

            #Request Simpy Resource from Richmond
            with richmond.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and richmond.pallet_reserve_t1.level > 0:
                    yield richmond.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and richmond.pallet_reserve_t2.level > 0:
                    yield richmond.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and richmond.pallet_reserve_t3.level > 0:
                    yield richmond.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and richmond.pallet_reserve_t4.level > 0:
                    yield richmond.pallet_reserve_t4.get(pallets_sent_t4)

                yield richmond.pallet_reserve_t5.get(pallets_sent_t5)


                #Put pallet need into pallet need container
                #yield richmond.pallet_need.put(richmond_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Richmond to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Richmond to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Richmond can fulfill with current total pallet level
        fillable_demand = richmond.pallet_reserve_t1.level + richmond.pallet_reserve_t2.level + richmond.pallet_reserve_t3.level + richmond.pallet_reserve_t4.level + richmond.pallet_reserve_t5.level
 
        #Request Simpy Resource from Richmond
        with richmond.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield richmond.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield richmond.pallet_need.get(pallets_sent - fillable_demand)

        
        #Re-run this process, sending pallets from Richmond to RDC
        yield env.process(richmond_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Richmond Return

In [227]:
def dsd_return_richmond(env, richmond):
    '''This process returns pallets back to Richmond'''

    #Locally initiate pallet availability containers
    global pallets_available_to_return_t2
    global pallets_available_to_return_t3
    global pallets_available_to_return_t4
    global pallets_available_to_return_t5

    #Indicate destination for model output
    destination = 'Richmond'

    #Get current avaialable to return pallet level for each turn container
    curr_pallets_t2 = pallets_available_to_return_t2
    curr_pallets_t3 = pallets_available_to_return_t3
    curr_pallets_t4 = pallets_available_to_return_t4
    curr_pallets_t5 = pallets_available_to_return_t5
    

    #Assign current Richmond Hub pallet need for use in formulas
    richmond_need = richmond.pallet_need.level
    print('Time', env.now, 'richmond_need', richmond.pallet_need.level)

    #Check if Richmond has any pallet need in current time step
    if richmond_need > 0:

        #Starting in reverse order, check if there is enough turn 5 availability to cover Richmond pallet need
        if curr_pallets_t5 >= richmond_need:
            #If so, take Richmond need pallets out of available to return container
            pallets_available_to_return_t5 -= richmond_need

            #Request Simpy Resource from Richmond
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Richmond
                yield richmond.pallet_reserve_intransit.put(richmond_need)

            #Pallets in transit from DSD back to Richmond
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield richmond.pallet_reserve_intransit.get(richmond_need)
                
                #Put current Richmond Hub pallet need back into Richmond Hub pallet reserve container once returned
                yield richmond.pallet_reserve_t5.put(richmond_need)

                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += richmond_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': richmond_need})

                #Take pallet need out of Richmond Hub once returned
                #yield richmond.pallet_need.get(richmond_need)


        #Check if T4 + T5 pallets can cover Richmond need
        elif curr_pallets_t5 + curr_pallets_t4 >= richmond_need:
            #All (if any) T5 pallets will be returned
            richmond_need_t5 = curr_pallets_t5
            #Calculate T4 pallets that will be sent
            richmond_need_t4 = richmond_need - curr_pallets_t5

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 -= richmond_need_t4


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req
                #Flag pallets are on their way back to Richmond
                yield richmond.pallet_reserve_intransit.put(richmond_need)

            #Pallets in transit from DSD back to Richmond
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield richmond.pallet_reserve_intransit.get(richmond_need)

                #Put current Richmond Hub pallet need back into Richmond Hub pallet reserve container once returned
                if richmond_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Richmond T5 container
                    yield richmond.pallet_reserve_t5.put(richmond_need_t5)

                if richmond_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Richmond T4 container
                    yield richmond.pallet_reserve_t4.put(richmond_need_t4)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += richmond_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': richmond_need})

                #Take pallet need out of Richmond container once pallets are returned
                #yield richmond.pallet_need.get(richmond_need)

        #Check if T3 + T4 + T5 pallets can cover Richmond need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 >= richmond_need:
            #All (if any) T5 pallets will be returned
            richmond_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            richmond_need_t4 = curr_pallets_t4
            #Calculate T3 pallets that will be sent
            richmond_need_t3 = richmond_need - curr_pallets_t5 - curr_pallets_t4

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 -= richmond_need_t3


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Richmond
                yield richmond.pallet_reserve_intransit.put(richmond_need)

            #Pallets in transit from DSD back to Richmond
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield richmond.pallet_reserve_intransit.get(richmond_need)

                #Put current Richmond Hub pallet need back into Richmond Hub pallet reserve container once returned
                if richmond_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Richmond T5 container
                    yield richmond.pallet_reserve_t5.put(richmond_need_t5)

                if richmond_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Richmond T4 container
                    yield richmond.pallet_reserve_t4.put(richmond_need_t4)

                if richmond_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Richmond T3 container
                    yield richmond.pallet_reserve_t3.put(richmond_need_t3)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += richmond_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': richmond_need})

                #Take pallet need out of Richmond container once pallets are returned
                #yield richmond.pallet_need.get(richmond_need)


        #Check if T2 + T3 + T4 + T5 pallets can cover Richmond need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 >= richmond_need:
            print('Returning pallets to richmond')
            #All (if any) T5 pallets will be returned
            richmond_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            richmond_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            richmond_need_t3 = curr_pallets_t3
            #Calculate T2 pallets that will be sent
            richmond_need_t2 = richmond_need - curr_pallets_t5 - curr_pallets_t4 - curr_pallets_t3

            print('TIME', env.now, 'pallets t2 BEFORE', pallets_available_to_return_t2)
            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t2 -= richmond_need_t2
            print('TIME RICHMOND', env.now, 'need', richmond_need)
            print('TIME', env.now, 't2 need', richmond_need_t2)
            print('TIME', env.now, 't3 need', richmond_need_t3)
            print('TIME', 'pallets t2', pallets_available_to_return_t2)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Richmond
                yield richmond.pallet_reserve_intransit.put(richmond_need)

            #Pallets in transit from DSD back to Richmond Hub
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield richmond.pallet_reserve_intransit.get(richmond_need)

                #Put current Richmond Hub pallet need back into Richmond Hub pallet reserve container once returned
                if richmond_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Richmond T5 container
                    yield richmond.pallet_reserve_t5.put(richmond_need_t5)

                if richmond_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Richmond T4 container
                    yield richmond.pallet_reserve_t4.put(richmond_need_t4)

                if richmond_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Richmond T3 container
                    yield richmond.pallet_reserve_t3.put(richmond_need_t3)

                if richmond_need_t2 > 0:
                    #Put pallets returned from T2 PATR container back into Richmond T2 container
                    yield richmond.pallet_reserve_t2.put(richmond_need_t2)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += richmond_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': richmond_need})
                
                #Take pallet need out of Richmond container once pallets are returned
                #yield richmond.pallet_need.get(richmond_need)


        #Can't fill full pallet need in Richmond with pallet return program, send everything that is available
        else:
            #All (if any) T5 pallets will be returned
            richmond_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            richmond_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            richmond_need_t3 = curr_pallets_t3
            #All (if any) T2 pallets will be returned
            richmond_need_t2 = curr_pallets_t2

            print('Returning ALL LEFTOVER pallets to richmond')
            print('TIME', env.now, 'pallets t2 BEFORE', pallets_available_to_return_t2)
            #Adjust pallets available to return containers
            pallets_available_to_return_t2 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t5 = 0
            print('TIME RICHMOND', env.now, 'need', richmond_need)
            print('TIME', env.now, 't2 need', richmond_need_t2)
            print('TIME', env.now, 't3 need', richmond_need_t3)
            print('TIME', 'pallets t2', pallets_available_to_return_t2)
            
            #Check to make sure there are any pallets available to return
            if curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 > 0:
                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Flag pallets are on their way back to Richmond
                    yield richmond.pallet_reserve_intransit.put(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                #Pallets in transit from DSD back to Richmond Hub
                yield env.timeout(END_TO_PLANT)

                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Take out pallets in transit as they have arrived at the facility
                    yield richmond.pallet_reserve_intransit.get(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                    #Put current Richmond Hub pallet need back into Richmond Hub pallet reserve container once returned
                    if richmond_need_t5 > 0:
                        #Put pallets returned from T5 PATR container back into Richmond T5 container
                        yield richmond.pallet_reserve_t5.put(richmond_need_t5)

                    if richmond_need_t4 > 0:
                        #Put pallets returned from T4 PATR container back into Richmond T4 container
                        yield richmond.pallet_reserve_t4.put(richmond_need_t4)

                    if richmond_need_t3 > 0:
                        #Put pallets returned from T3 PATR container back into Richmond T3 container
                        yield richmond.pallet_reserve_t3.put(richmond_need_t3)

                    if richmond_need_t2 > 0:
                        #Put pallets returned from T2 PATR container back into Richmond T2 container
                        yield richmond.pallet_reserve_t2.put(richmond_need_t2)
                    
                    #Floor the simulation day to calculate current week's return
                    week = env.now//7
                    weekly_return_counter[week] += (richmond_need_t2 + richmond_need_t3 + richmond_need_t4 + richmond_need_t5)

                    #Add weekly return by location for dashboard use
                    return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': (richmond_need_t2 + richmond_need_t3 + richmond_need_t4 + richmond_need_t5)})

                    #Take pallet need out of Richmond container once pallets are returned
                    #yield richmond.pallet_need.get(richmond_need_t2 + richmond_need_t3 + richmond_need_t4 + richmond_need_t5)


            #No pallets avaialable left to return in the current time step
            else:
                print('No pallets available to return in Richmond')

---

### Portland

In [228]:
#Group Portland outbound data by branch type to use for lookups in each outbound process function
portland_outbound_grouped = portland_outbound.groupby(['Operational Week', 'Dest Plant Type'])['Picked Pallets'].sum().reset_index()

Portland to DSD

In [229]:
#Input Portland actual outbound to DSDs in list to index in processes
portland_dsd_weekly_outbound = list(portland_outbound_grouped[portland_outbound_grouped['Dest Plant Type'] == 'Branch']['Picked Pallets'].values)

#Extrapolate demand out a year
portland_dsd_weekly_outbound = portland_dsd_weekly_outbound + portland_dsd_weekly_outbound

In [230]:
def portland_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Portland plant to dsd node'''

    print('time', env.now, 'portland_to_dsd running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Portland to DSD
    pallets_sent = portland_dsd_weekly_outbound[week - 1]

    #Grab current pallet reserve container levels for Portland
    curr_t1_level = portland.pallet_reserve_t1.level
    curr_t2_level = portland.pallet_reserve_t2.level
    curr_t3_level = portland.pallet_reserve_t3.level
    curr_t4_level = portland.pallet_reserve_t4.level
    curr_t5_level = portland.pallet_reserve_t5.level

    #Check if Portland pallet reserve T1 pallets can cover weekly demand
    if portland.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Portland Hub
        with portland.capacity.request() as req:
            yield req

            #Take out weekly pallet demand from T1 pallet supply
            yield portland.pallet_reserve_t1.get(pallets_sent)

            #Check if there is pallet need, if so set back to 0
            if portland.pallet_need.level > 0:
                yield portland.pallet_need.get(portland.pallet_need.level)
            
            #Get following weeks outbound demand to use as pallet need
            yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])


            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Portland to DSD
            yield env.timeout(PLANTS_TO_DSD)

        #Run next step in the model, sending pallets from Portland to DSD Branches
        yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Second check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level >= pallets_sent:
        #Check if Portland pallet reserve T2 pallets can cover weekly demand
        if portland.pallet_reserve_t2.level >= pallets_sent:
            
            #If true, only T2 pallets will be used
            pallets_sent_t1 = 0

            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(portland.pallet_reserve_t2.level - pallets_sent) < 0.01:
                pallets_sent_t2 = portland.pallet_reserve_t2.level
            else:
                pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)


                #Put pallet need into pallet need container
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use both T1 + T2 pallet supply to cover demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)


                yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)

                #Put pallet need into pallet need container
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Third check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Portland pallet reserve T3 pallets can cover weekly demand
        if curr_t3_level >= pallets_sent:

            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)
                    
                #Put pallet need into pallet need container
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2 & T3 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - portland.pallet_reserve_t2.level <= 0:
                #If true, no T1 pallets will be sent
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = portland.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t2.level - portland.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = portland.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - portland.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = portland.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and portland.pallet_reserve_t2.level > 0:
                    yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and portland.pallet_reserve_t3.level > 0:
                    yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)

                #Put pallet need into pallet need container
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fourth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level + portland.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Portland pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield portland.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)

                #Put pallet need into pallet need conatiner
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - portland.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = portland.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T2, T3 & T4 pallets to cover weekly demand
            elif pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = portland.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - portland.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level - portland.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = portland.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = portland.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and portland.pallet_reserve_t2.level > 0:
                    yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and portland.pallet_reserve_t3.level > 0:
                    yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                if portland.pallet_reserve_t4.level > 0:
                    yield portland.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)

                #Put pallet need into pallet need container
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fifth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level + portland.pallet_reserve_t4.level + portland.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Portland pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield portland.pallet_reserve_t5.get(pallets_sent_t5)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)

                #Put pallet need into pallet need container
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - portland.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level < 0.01):
                    pallets_sent_t4 = portland.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = curr_t5_level
            
            #Must use mix of T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = portland.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - portland.pallet_reserve_t4.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = portland.pallet_reserve_t5.level

            #Must use mix of T2, T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = portland.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = portland.pallet_reserve_t5.level
            
            #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level - portland.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = portland.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = portland.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = portland.pallet_reserve_t5.level
        
            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and portland.pallet_reserve_t2.level > 0:
                    yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and portland.pallet_reserve_t3.level > 0:
                    yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and portland.pallet_reserve_t4.level > 0:
                    yield portland.pallet_reserve_t4.get(pallets_sent_t4)

                yield portland.pallet_reserve_t5.get(pallets_sent_t5)

                #Check if there is pallet need, if so set back to 0
                if portland.pallet_need.level > 0:
                    yield portland.pallet_need.get(portland.pallet_need.level)

                #Put pallet need into pallet need container
                yield portland.pallet_need.put(portland_dsd_weekly_outbound[week + 2] + portland_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Portland to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Portland to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Portland can fulfill with current total pallet level
        fillable_demand = portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level + portland.pallet_reserve_t4.level + portland.pallet_reserve_t5.level

        #Request Simpy Resource from Portland
        with portland.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield portland.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield portland.pallet_need.get(pallets_sent - fillable_demand)
        
        #Re-run this process, sending pallets from Portland to DSD
        yield env.process(portland_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Portland to RDC

In [231]:
#Input Portland actual outbound to RDCs in list to index in processes
portland_rdc_weekly_outbound = list(portland_outbound_grouped[portland_outbound_grouped['Dest Plant Type'] == 'RDC']['Picked Pallets'].values)

#Extrapolate demand out a year
portland_rdc_weekly_outbound = portland_rdc_weekly_outbound + portland_rdc_weekly_outbound

In [232]:
def portland_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Portland plant to RDC node'''

    print('time', env.now, 'portland_to_rdc running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Portland to DSD
    pallets_sent = portland_rdc_weekly_outbound[week - 1]

    #Grab current pallet reserve container levels for Portland
    curr_t1_level = portland.pallet_reserve_t1.level
    curr_t2_level = portland.pallet_reserve_t2.level
    curr_t3_level = portland.pallet_reserve_t3.level
    curr_t4_level = portland.pallet_reserve_t4.level
    curr_t5_level = portland.pallet_reserve_t5.level

    #Check if Portland pallet reserve T1 pallets can cover weekly demand
    if portland.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Portland
        with portland.capacity.request() as req:
            yield req

            #Take pallets out of respective turn conatiner
            yield portland.pallet_reserve_t1.get(pallets_sent)

            #Put pallet need into pallet need container
            #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])


            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Richmon to RDCs
            yield env.timeout(PLANTS_TO_RDC)

        #Run next step in the model, sending pallets from the Portland to RDCs
        yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level >= pallets_sent:
        #Check if Portland pallet reserve T2 pallets can cover weekly demand
        if curr_t2_level >= pallets_sent:
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = 0
            pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                #Put pallet need into pallet need container
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)

                yield portland.pallet_reserve_t2.get(portland.pallet_reserve_t2.level)

                #Put pallet need into pallet need conatiner
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Portland pallet reserve T3 pallets can cover weekly demand
        if portland.pallet_reserve_t3.level >= pallets_sent:
            
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(portland.pallet_reserve_t3.level - pallets_sent) < 0.0001:
                pallets_sent_t3 = portland.pallet_reserve_t3.level
            else:
                pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - portland.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = portland.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0
            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t2.level - portland.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = portland.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - portland.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = portland.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and portland.pallet_reserve_t2.level > 0:
                    yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and portland.pallet_reserve_t3.level > 0:
                    yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from the Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level + portland.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Portland pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield portland.pallet_reserve_t4.get(pallets_sent_t4)

                #Put pallet need into pallet need container
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - portland.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = portland.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 pallets
            elif pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = portland.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - portland.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level - portland.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = portland.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = portland.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and portland.pallet_reserve_t2.level > 0:
                    yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and portland.pallet_reserve_t3.level > 0:
                    yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                yield portland.pallet_reserve_t4.get(pallets_sent_t4)

                #Put pallet need into pallet need container
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level + portland.pallet_reserve_t4.level + portland.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Portland pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:

            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield portland.pallet_reserve_t5.get(pallets_sent_t5)

                #Put pallet need into pallet need container
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - portland.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - portland.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level) < 0.01:
                    pallets_sent_t4 = portland.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                pallets_sent_t5 = curr_t5_level
            
            #Check to see if weekly demand can be covered only using T3 + T4 + T5 pallets
            elif pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = portland.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - portland.pallet_reserve_t4.level 
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = portland.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 + T5 pallets
            elif pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = portland.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = portland.pallet_reserve_t5.level
            
            #Must use a mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level - portland.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = portland.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - portland.pallet_reserve_t4.level - portland.pallet_reserve_t3.level - portland.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = portland.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = portland.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = portland.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = portland.pallet_reserve_t5.level

            #Request Simpy Resource from Portland
            with portland.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and portland.pallet_reserve_t1.level > 0:
                    yield portland.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and portland.pallet_reserve_t2.level > 0:
                    yield portland.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and portland.pallet_reserve_t3.level > 0:
                    yield portland.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and portland.pallet_reserve_t4.level > 0:
                    yield portland.pallet_reserve_t4.get(pallets_sent_t4)

                yield portland.pallet_reserve_t5.get(pallets_sent_t5)

                #Put pallet need into pallet need container
                #yield portland.pallet_need.put(portland_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Portland to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Portland to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Portland can fulfill with current total pallet level
        fillable_demand = portland.pallet_reserve_t1.level + portland.pallet_reserve_t2.level + portland.pallet_reserve_t3.level + portland.pallet_reserve_t4.level + portland.pallet_reserve_t5.level
 
        #Request Simpy Resource from Portland
        with portland.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield portland.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield portland.pallet_need.get(pallets_sent - fillable_demand)

        
        #Re-run this process, sending pallets from Portland to RDC
        yield env.process(portland_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Portland Return

In [233]:
def dsd_return_portland(env, portland):
    '''This process returns pallets back to Portland'''

    #Locally initiate pallet availability containers
    global pallets_available_to_return_t2
    global pallets_available_to_return_t3
    global pallets_available_to_return_t4
    global pallets_available_to_return_t5

    #Indicate destination for model output
    destination = 'Portland'

    #Get current avaialable to return pallet level for each turn container
    curr_pallets_t2 = pallets_available_to_return_t2
    curr_pallets_t3 = pallets_available_to_return_t3
    curr_pallets_t4 = pallets_available_to_return_t4
    curr_pallets_t5 = pallets_available_to_return_t5
    

    #Assign current Portland Hub pallet need for use in formulas
    portland_need = portland.pallet_need.level

    #Check if Portland has any pallet need in current time step
    if portland_need > 0:

        #Starting in reverse order, check if there is enough turn 5 availability to cover Portland pallet need
        if curr_pallets_t5 >= portland_need:
            #If so, take Portland need pallets out of available to return container
            pallets_available_to_return_t5 -= portland_need

            #Request Simpy Resource from Portland
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Portland
                yield portland.pallet_reserve_intransit.put(portland_need)

            #Pallets in transit from DSD back to Portland
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield portland.pallet_reserve_intransit.get(portland_need)
                
                #Put current Portland Hub pallet need back into Portland Hub pallet reserve container once returned
                yield portland.pallet_reserve_t5.put(portland_need)

                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += portland_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 12, 'Expected Pallet Return': portland_need})

                #Take palllets out of Portland Hub once returned 
                #yield portland.pallet_need.get(portland_need)


        #Check if T4 + T5 pallets can cover Portland need
        elif curr_pallets_t5 + curr_pallets_t4 >= portland_need:
            #All (if any) T5 pallets will be returned
            portland_need_t5 = curr_pallets_t5
            #Calculate T4 pallets that will be sent
            portland_need_t4 = portland_need - curr_pallets_t5

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 -= portland_need_t4


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req
                #Flag pallets are on their way back to Portland
                yield portland.pallet_reserve_intransit.put(portland_need)

            #Pallets in transit from DSD back to Portland
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield portland.pallet_reserve_intransit.get(portland_need)

                #Put current Portland Hub pallet need back into Portland Hub pallet reserve container once returned
                if portland_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Portland T5 container
                    yield portland.pallet_reserve_t5.put(portland_need_t5)

                if portland_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Portland T4 container
                    yield portland.pallet_reserve_t4.put(portland_need_t4)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += portland_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': portland_need})

                #Take pallet need out of Portland container once pallets are returned
                #yield portland.pallet_need.get(portland_need)


        #Check if T3 + T4 + T5 pallets can cover Portland need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 >= portland_need:
            #All (if any) T5 pallets will be returned
            portland_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            portland_need_t4 = curr_pallets_t4
            #Calculate T3 pallets that will be sent
            portland_need_t3 = portland_need - curr_pallets_t5 - curr_pallets_t4

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 -= portland_need_t3


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Portland
                yield portland.pallet_reserve_intransit.put(portland_need)

            #Pallets in transit from DSD back to Portland
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield portland.pallet_reserve_intransit.get(portland_need)

                #Put current Portland Hub pallet need back into Portland Hub pallet reserve container once returned
                if portland_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Portland T5 container
                    yield portland.pallet_reserve_t5.put(portland_need_t5)

                if portland_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Portland T4 container
                    yield portland.pallet_reserve_t4.put(portland_need_t4)

                if portland_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Portland T3 container
                    yield portland.pallet_reserve_t3.put(portland_need_t3)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += portland_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': portland_need})

                #Take pallet need out of Portland container once pallets are returned
                #yield portland.pallet_need.get(portland_need)


        #Check if T2 + T3 + T4 + T5 pallets can cover Portland need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 >= portland_need:
            #All (if any) T5 pallets will be returned
            portland_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            portland_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            portland_need_t3 = curr_pallets_t3
            #Calculate T2 pallets that will be sent
            portland_need_t2 = portland_need - curr_pallets_t5 - curr_pallets_t4 - curr_pallets_t3
            print('TIME', env.now, 'pallets t2 BEFORE', pallets_available_to_return_t2)
            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t2 -= portland_need_t2

            print('TIME PORTlAND', env.now, 'need', portland_need)
            print('TIME', env.now, 't2 need', portland_need_t2)
            print('TIME', env.now, 't3 need', portland_need_t3)
            print('TIME', env.now, 'pallets t2', pallets_available_to_return_t2)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Portland
                yield portland.pallet_reserve_intransit.put(portland_need)

            #Pallets in transit from DSD back to Portland Hub
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield portland.pallet_reserve_intransit.get(portland_need)

                #Put current Portland Hub pallet need back into Portland Hub pallet reserve container once returned
                if portland_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Portland T5 container
                    yield portland.pallet_reserve_t5.put(portland_need_t5)

                if portland_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Portland T4 container
                    yield portland.pallet_reserve_t4.put(portland_need_t4)

                if portland_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Portland T3 container
                    yield portland.pallet_reserve_t3.put(portland_need_t3)

                if portland_need_t2 > 0:
                    #Put pallets returned from T2 PATR container back into Portland T2 container
                    yield portland.pallet_reserve_t2.put(portland_need_t2)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += portland_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': portland_need})
                
                #Take pallet need out of Portland container once pallets are returned
                #yield portland.pallet_need.get(portland_need)


        #Can't fill full pallet need in Portland with pallet return program, send everything that is available
        else:
            #All (if any) T5 pallets will be returned
            portland_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            portland_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            portland_need_t3 = curr_pallets_t3
            #All (if any) T2 pallets will be returned
            portland_need_t2 = curr_pallets_t2

            #Adjust pallets available to return containers
            pallets_available_to_return_t2 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t5 = 0

            #Check to make sure there are any pallets available to return
            if curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 > 0:
                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Flag pallets are on their way back to Portland
                    yield portland.pallet_reserve_intransit.put(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                #Pallets in transit from DSD back to Portland Hub
                yield env.timeout(END_TO_PLANT)

                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Take out pallets in transit as they have arrived at the facility
                    yield portland.pallet_reserve_intransit.get(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                    #Put current Portland Hub pallet need back into Portland Hub pallet reserve container once returned
                    if portland_need_t5 > 0:
                        #Put pallets returned from T5 PATR container back into Portland T5 container
                        yield portland.pallet_reserve_t5.put(portland_need_t5)

                    if portland_need_t4 > 0:
                        #Put pallets returned from T4 PATR container back into Portland T4 container
                        yield portland.pallet_reserve_t4.put(portland_need_t4)

                    if portland_need_t3 > 0:
                        #Put pallets returned from T3 PATR container back into Portland T3 container
                        yield portland.pallet_reserve_t3.put(portland_need_t3)

                    if portland_need_t2 > 0:
                        #Put pallets returned from T2 PATR container back into Portland T2 container
                        yield portland.pallet_reserve_t2.put(portland_need_t2)
                    
                    #Floor the simulation day to calculate current week's return
                    week = env.now//7
                    weekly_return_counter[week] += (portland_need_t2 + portland_need_t3 + portland_need_t4 + portland_need_t5)

                    #Add weekly return by location for dashboard use
                    return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': (portland_need_t2 + portland_need_t3 + portland_need_t4 + portland_need_t5)})

                    #Take pallet need out of Portland container once pallets are returned
                    #yield portland.pallet_need.get(portland_need_t2 + portland_need_t3 + portland_need_t4 + portland_need_t5)


            #No pallets avaialable left to return in the current time step
            else:
                print('No pallets available to return in Portland')

-----

### Chicago

In [234]:
#Group Portland outbound data by branch type to use for lookups in each outbound process function
chicago_outbound_grouped = chicago_outbound.groupby(['Operational Week', 'Dest Plant Type'])['Picked Pallets'].sum().reset_index()

Chicago to DSD

In [235]:
#Input Chicago actual outbound to DSDs in list to index in processes
chicago_dsd_weekly_outbound = list(chicago_outbound_grouped[chicago_outbound_grouped['Dest Plant Type'] == 'Branch']['Picked Pallets'].values)

#Extrapolate demand out a year
chicago_dsd_weekly_outbound = chicago_dsd_weekly_outbound + chicago_dsd_weekly_outbound

In [236]:
def chicago_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Chicago plant to dsd node'''

    print('time', env.now, 'chicago-to_dsd running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Chicago to DSD
    pallets_sent = chicago_dsd_weekly_outbound[week - 1]

    #Grab current pallet reserve container levels for Chicago
    curr_t1_level = chicago.pallet_reserve_t1.level
    curr_t2_level = chicago.pallet_reserve_t2.level
    curr_t3_level = chicago.pallet_reserve_t3.level
    curr_t4_level = chicago.pallet_reserve_t4.level
    curr_t5_level = chicago.pallet_reserve_t5.level

    #Check if Chicago pallet reserve T1 pallets can cover weekly demand
    if chicago.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Chicago Hub
        with chicago.capacity.request() as req:
            yield req

            #Take out weekly pallet demand from T1 pallet supply
            yield chicago.pallet_reserve_t1.get(pallets_sent)

            #Check if there is pallet need, if so set back to 0
            if chicago.pallet_need.level > 0:
                yield chicago.pallet_need.get(chicago.pallet_need.level)

            #Get following weeks outbound demand to use as pallet need
            yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])


            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Chicago to DSD
            yield env.timeout(PLANTS_TO_DSD)

        #Run next step in the model, sending pallets from Chicago to DSD Branches
        yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Second check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level >= pallets_sent:
        #Check if Chicago pallet reserve T2 pallets can cover weekly demand
        if chicago.pallet_reserve_t2.level >= pallets_sent:
            
            #If true, only T2 pallets will be used
            pallets_sent_t1 = 0

            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(chicago.pallet_reserve_t2.level - pallets_sent) < 0.01:
                pallets_sent_t2 = chicago.pallet_reserve_t2.level
            else:
                pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need container
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use both T1 + T2 pallet supply to cover demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)


                yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need container
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Third check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Chicago pallet reserve T3 pallets can cover weekly demand
        if curr_t3_level >= pallets_sent:

            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need container
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2 & T3 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - chicago.pallet_reserve_t2.level <= 0:
                #If true, no T1 pallets will be sent
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = chicago.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t2.level - chicago.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = chicago.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - chicago.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = chicago.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and chicago.pallet_reserve_t2.level > 0:
                    yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and chicago.pallet_reserve_t3.level > 0:
                    yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need container
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fourth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level + chicago.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Chicago pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield chicago.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need conatiner
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - chicago.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = chicago.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T2, T3 & T4 pallets to cover weekly demand
            elif pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = chicago.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - chicago.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level - chicago.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = chicago.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = chicago.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and chicago.pallet_reserve_t2.level > 0:
                    yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and chicago.pallet_reserve_t3.level > 0:
                    yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                if chicago.pallet_reserve_t4.level > 0:
                    yield chicago.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need container
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)
            
            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Fifth check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level + chicago.pallet_reserve_t4.level + chicago.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Chicago pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield chicago.pallet_reserve_t5.get(pallets_sent_t5)

                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need container
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - chicago.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level < 0.01):
                    pallets_sent_t4 = chicago.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = curr_t5_level
            
            #Must use mix of T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = chicago.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - chicago.pallet_reserve_t4.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = chicago.pallet_reserve_t5.level

            #Must use mix of T2, T3, T4 & T5 pallets to cover weekly demand
            elif pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = chicago.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = chicago.pallet_reserve_t5.level
            
            #Must use mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level - chicago.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = chicago.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = chicago.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = chicago.pallet_reserve_t5.level
        
            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and chicago.pallet_reserve_t2.level > 0:
                    yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and chicago.pallet_reserve_t3.level > 0:
                    yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and chicago.pallet_reserve_t4.level > 0:
                    yield chicago.pallet_reserve_t4.get(pallets_sent_t4)

                yield chicago.pallet_reserve_t5.get(pallets_sent_t5)


                #Check if there is pallet need, if so set back to 0
                if chicago.pallet_need.level > 0:
                    yield chicago.pallet_need.get(chicago.pallet_need.level)

                #Put pallet need into pallet need container
                yield chicago.pallet_need.put(chicago_dsd_weekly_outbound[week + 2] + chicago_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Chicago to DSD
                yield env.timeout(PLANTS_TO_DSD)

            #Run next step in the model, sending pallets from the Chicago to DSD Branches
            yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Chicago can fulfill with current total pallet level
        fillable_demand = chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level + chicago.pallet_reserve_t4.level + chicago.pallet_reserve_t5.level

        #Request Simpy Resource from Chicago
        with chicago.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield chicago.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield chicago.pallet_need.get(pallets_sent - fillable_demand)
        
        #Re-run this process, sending pallets from Chicago to DSD
        yield env.process(chicago_to_dsd(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Chicago to RDC

In [237]:
#Input Chicago actual outbound to RDCs in list to index in processes
chicago_rdc_weekly_outbound = list(chicago_outbound_grouped[chicago_outbound_grouped['Dest Plant Type'] == 'RDC']['Picked Pallets'].values)

#Extrapolate demand out a year
chicago_rdc_weekly_outbound = chicago_rdc_weekly_outbound + chicago_rdc_weekly_outbound

In [238]:
def chicago_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Chicago plant to RDC node'''

    print('time', env.now, 'chicago_to_rdc running')

    #Map current week based on current Simpy day
    try:
        week = env.now//7
    except:
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Chicago to DSD
    pallets_sent = chicago_rdc_weekly_outbound[week - 1]

    #Grab current pallet reserve container levels for Chicago
    curr_t1_level = chicago.pallet_reserve_t1.level
    curr_t2_level = chicago.pallet_reserve_t2.level
    curr_t3_level = chicago.pallet_reserve_t3.level
    curr_t4_level = chicago.pallet_reserve_t4.level
    curr_t5_level = chicago.pallet_reserve_t5.level

    #Check if Chicago pallet reserve T1 pallets can cover weekly demand
    if chicago.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Chicago
        with chicago.capacity.request() as req:
            yield req

            #Take pallets out of respective turn conatiner
            yield chicago.pallet_reserve_t1.get(pallets_sent)

            #Put pallet need into pallet need container
            #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])


            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Chicago to RDCs
            yield env.timeout(PLANTS_TO_RDC)

        #Run next step in the model, sending pallets from the Chicago to RDCs
        yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level >= pallets_sent:
        #Check if Chicago pallet reserve T2 pallets can cover weekly demand
        if curr_t2_level >= pallets_sent:
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = 0
            pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                #Put pallet need into pallet need container
                #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and chicago.pallet_reserve_t2.level > 0:
                    yield chicago.pallet_reserve_t2.get(chicago.pallet_reserve_t2.level)

                #Put pallet need into pallet need conatiner
                #if pallets_sent_t3 > 0 and chicago.pallet_reserve_t3.level > 0:
                    #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Chicago pallet reserve T3 pallets can cover weekly demand
        if chicago.pallet_reserve_t3.level >= pallets_sent:
            
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(chicago.pallet_reserve_t3.level - pallets_sent) < 0.0001:
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
            else:
                pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - chicago.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = chicago.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0
            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t2.level - chicago.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = chicago.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - chicago.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = chicago.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and chicago.pallet_reserve_t2.level > 0:
                    yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and chicago.pallet_reserve_t3.level > 0:
                    yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                #Put pallet need into pallet need conatiner
                #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from the Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level + chicago.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Chicago pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield chicago.pallet_reserve_t4.get(pallets_sent_t4)

                #Put pallet need into pallet need container
                #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - chicago.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = chicago.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 pallets
            elif pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = chicago.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - chicago.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level - chicago.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = chicago.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = chicago.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and chicago.pallet_reserve_t2.level > 0:
                    yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and chicago.pallet_reserve_t3.level > 0:
                    yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                yield chicago.pallet_reserve_t4.get(pallets_sent_t4)

                #Put pallet need into pallet need container
                #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level + chicago.pallet_reserve_t4.level + chicago.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Chicago pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:

            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield chicago.pallet_reserve_t5.get(pallets_sent_t5)

                #Put pallet need into pallet need container
                #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - chicago.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - chicago.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level) < 0.01:
                    pallets_sent_t4 = chicago.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                pallets_sent_t5 = curr_t5_level
            
            #Check to see if weekly demand can be covered only using T3 + T4 + T5 pallets
            elif pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = chicago.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - chicago.pallet_reserve_t4.level 
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = chicago.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 + T5 pallets
            elif pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = chicago.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = chicago.pallet_reserve_t5.level
            
            #Must use a mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level - chicago.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = chicago.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - chicago.pallet_reserve_t4.level - chicago.pallet_reserve_t3.level - chicago.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = chicago.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = chicago.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = chicago.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = chicago.pallet_reserve_t5.level

            #Request Simpy Resource from Chicago
            with chicago.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and chicago.pallet_reserve_t1.level > 0:
                    yield chicago.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and chicago.pallet_reserve_t2.level > 0:
                    yield chicago.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and chicago.pallet_reserve_t3.level > 0:
                    yield chicago.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and chicago.pallet_reserve_t4.level > 0:
                    yield chicago.pallet_reserve_t4.get(pallets_sent_t4)

                yield chicago.pallet_reserve_t5.get(pallets_sent_t5)

                #Put pallet need into pallet need container
                #yield chicago.pallet_need.put(chicago_rdc_weekly_outbound[week + 1])

                #Pallets in transit from Chicago to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Chicago to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Chicago can fulfill with current total pallet level
        fillable_demand = chicago.pallet_reserve_t1.level + chicago.pallet_reserve_t2.level + chicago.pallet_reserve_t3.level + chicago.pallet_reserve_t4.level + chicago.pallet_reserve_t5.level
 
        #Request Simpy Resource from Chicago
        with chicago.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield chicago.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield chicago.pallet_need.get(pallets_sent - fillable_demand)

        
        #Re-run this process, sending pallets from Chicago to RDC
        yield env.process(chicago_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Chicago Return

In [239]:
def dsd_return_chicago(env, chicago):
    '''This process returns pallets back to Chicago'''

    #Locally initiate pallet availability containers
    global pallets_available_to_return_t2
    global pallets_available_to_return_t3
    global pallets_available_to_return_t4
    global pallets_available_to_return_t5

    #Indicate destination for model output
    destination = 'Chicago'

    #Get current avaialable to return pallet level for each turn container
    curr_pallets_t2 = pallets_available_to_return_t2
    curr_pallets_t3 = pallets_available_to_return_t3
    curr_pallets_t4 = pallets_available_to_return_t4
    curr_pallets_t5 = pallets_available_to_return_t5
    

    #Assign current Chicago Hub pallet need for use in formulas
    chicago_need = chicago.pallet_need.level

    #Check if Chicago has any pallet need in current time step
    if chicago_need > 0:

        #Starting in reverse order, check if there is enough turn 5 availability to cover Chicago pallet need
        if curr_pallets_t5 >= chicago_need:
            #If so, take Chicago need pallets out of available to return container
            pallets_available_to_return_t5 -= chicago_need

            #Request Simpy Resource from Chicago
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Chicago
                yield chicago.pallet_reserve_intransit.put(chicago_need)

            #Pallets in transit from DSD back to Chicago
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield chicago.pallet_reserve_intransit.get(chicago_need)
                
                #Put current Chicago Hub pallet need back into Chicago Hub pallet reserve container once returned
                yield chicago.pallet_reserve_t5.put(chicago_need)

                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += chicago_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': chicago_need})
                
                #Take pallets out of Chicago Hub once returned
                #yield chicago.pallet_need.get(chicago_need)


        #Check if T4 + T5 pallets can cover Chicago need
        elif curr_pallets_t5 + curr_pallets_t4 >= chicago_need:
            #All (if any) T5 pallets will be returned
            chicago_need_t5 = curr_pallets_t5
            #Calculate T4 pallets that will be sent
            chicago_need_t4 = chicago_need - curr_pallets_t5

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 -= chicago_need_t4


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req
                #Flag pallets are on their way back to Chicago
                yield chicago.pallet_reserve_intransit.put(chicago_need)

            #Pallets in transit from DSD back to Chicago
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield chicago.pallet_reserve_intransit.get(chicago_need)

                #Put current Chicago Hub pallet need back into Chicago Hub pallet reserve container once returned
                if chicago_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Chicago T5 container
                    yield chicago.pallet_reserve_t5.put(chicago_need_t5)

                if chicago_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Chicago T4 container
                    yield chicago.pallet_reserve_t4.put(chicago_need_t4)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += chicago_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': chicago_need})

                #Take pallet need out of Chicago container once pallets are returned
                #yield chicago.pallet_need.get(chicago_need)


        #Check if T3 + T4 + T5 pallets can cover Chicago need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 >= chicago_need:
            #All (if any) T5 pallets will be returned
            chicago_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            chicago_need_t4 = curr_pallets_t4
            #Calculate T3 pallets that will be sent
            chicago_need_t3 = chicago_need - curr_pallets_t5 - curr_pallets_t4

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 -= chicago_need_t3


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Chicago
                yield chicago.pallet_reserve_intransit.put(chicago_need)

            #Pallets in transit from DSD back to Chicago
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield chicago.pallet_reserve_intransit.get(chicago_need)

                #Put current Chicago Hub pallet need back into Chicago Hub pallet reserve container once returned
                if chicago_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Chicago T5 container
                    yield chicago.pallet_reserve_t5.put(chicago_need_t5)

                if chicago_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Chicago T4 container
                    yield chicago.pallet_reserve_t4.put(chicago_need_t4)

                if chicago_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Chicago T3 container
                    yield chicago.pallet_reserve_t3.put(chicago_need_t3)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += chicago_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': chicago_need})

                #Take pallet need out of Chicago container once pallets are returned
                #yield chicago.pallet_need.get(chicago_need)


        #Check if T2 + T3 + T4 + T5 pallets can cover Chicago need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 >= chicago_need:
            #All (if any) T5 pallets will be returned
            chicago_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            chicago_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            chicago_need_t3 = curr_pallets_t3
            #Calculate T2 pallets that will be sent
            chicago_need_t2 = chicago_need - curr_pallets_t5 - curr_pallets_t4 - curr_pallets_t3
            print('TIME', env.now, 'pallets t2 BEFORE', pallets_available_to_return_t2)
            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t2 -= chicago_need_t2

            print('TIME CHICAGO', env.now, 'need', chicago_need)
            print('TIME', env.now, 't2 need', chicago_need_t2)
            print('TIME', env.now, 't3 need', chicago_need_t3)
            print('TIME', env.now, 'pallets t2', pallets_available_to_return_t2)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Chicago
                yield chicago.pallet_reserve_intransit.put(chicago_need)

            #Pallets in transit from DSD back to Chicago Hub
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield chicago.pallet_reserve_intransit.get(chicago_need)

                #Put current Chicago Hub pallet need back into Chicago Hub pallet reserve container once returned
                if chicago_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Chicago T5 container
                    yield chicago.pallet_reserve_t5.put(chicago_need_t5)

                if chicago_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Chicago T4 container
                    yield chicago.pallet_reserve_t4.put(chicago_need_t4)

                if chicago_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Chicago T3 container
                    yield chicago.pallet_reserve_t3.put(chicago_need_t3)

                if chicago_need_t2 > 0:
                    #Put pallets returned from T2 PATR container back into Chicago T2 container
                    yield chicago.pallet_reserve_t2.put(chicago_need_t2)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += chicago_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': chicago_need})
            
                #Take pallet need out of Chicago container once pallets are returned
                #yield chicago.pallet_need.get(chicago_need)


        #Can't fill full pallet need in Chicago with pallet return program, send everything that is available
        else:
            #All (if any) T5 pallets will be returned
            chicago_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            chicago_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            chicago_need_t3 = curr_pallets_t3
            #All (if any) T2 pallets will be returned
            chicago_need_t2 = curr_pallets_t2

            #Adjust pallets available to return containers
            pallets_available_to_return_t2 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t5 = 0

            #Check to make sure there are any pallets available to return
            if curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 > 0:
                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Flag pallets are on their way back to Chicago
                    yield chicago.pallet_reserve_intransit.put(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                #Pallets in transit from DSD back to Chicago Hub
                yield env.timeout(END_TO_PLANT)

                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Take out pallets in transit as they have arrived at the facility
                    yield chicago.pallet_reserve_intransit.get(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                    #Put current Chicago Hub pallet need back into Chicago Hub pallet reserve container once returned
                    if chicago_need_t5 > 0:
                        #Put pallets returned from T5 PATR container back into Chicago T5 container
                        yield chicago.pallet_reserve_t5.put(chicago_need_t5)

                    if chicago_need_t4 > 0:
                        #Put pallets returned from T4 PATR container back into Chicago T4 container
                        yield chicago.pallet_reserve_t4.put(chicago_need_t4)

                    if chicago_need_t3 > 0:
                        #Put pallets returned from T3 PATR container back into Chicago T3 container
                        yield chicago.pallet_reserve_t3.put(chicago_need_t3)

                    if chicago_need_t2 > 0:
                        #Put pallets returned from T2 PATR container back into Chicago T2 container
                        yield chicago.pallet_reserve_t2.put(chicago_need_t2)
                    
                    #Floor the simulation day to calculate current week's return
                    week = env.now//7
                    weekly_return_counter[week] += (chicago_need_t2 + chicago_need_t3 + chicago_need_t4 + chicago_need_t5)

                    #Add weekly return by location for dashboard use
                    return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': (chicago_need_t2 + chicago_need_t3 + chicago_need_t4 + chicago_need_t5)})

                    #Take pallet need out of Chicago container once pallets are returned
                    #yield chicago.pallet_need.get(chicago_need_t2 + chicago_need_t3 + chicago_need_t4 + chicago_need_t5)


            #No pallets avaialable left to return in the current time step
            else:
                print('No pallets available to return in Chicago')

-----

### Naperville

In [240]:
#Group Portland outbound data by branch type to use for lookups in each outbound process function
naperville_outbound_grouped = naperville_outbound.groupby(['Operational Week', 'Dest Plant Type'])['Picked Pallets'].sum().reset_index()

Naperville to RDC

In [241]:
#Input Naperville actual outbound to RDCs in list to index in processes
naperville_rdc_weekly_outbound = list(naperville_outbound_grouped[naperville_outbound_grouped['Dest Plant Type'] == 'RDC']['Picked Pallets'].values)

#Extrapolate demand out a year
naperville_rdc_weekly_outbound = naperville_rdc_weekly_outbound + naperville_rdc_weekly_outbound

In [242]:
def naperville_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from Naperville plant to RDC node'''

    print('time', env.now, 'naperville_to_rdc running')
    
    #Map current week based on current Simpy day
    try:
        week = env.now // 7
    except Exception as e:
        print(e)
        print('Not running process on the correct day - look at simulation timing')

    #Get current weeks actual outoubnd volume from Naperville to RDC
    pallets_sent = naperville_rdc_weekly_outbound[week - 1]

    #Grab current pallet reserve container levels for Naperville
    curr_t1_level = naperville.pallet_reserve_t1.level
    curr_t2_level = naperville.pallet_reserve_t2.level
    curr_t3_level = naperville.pallet_reserve_t3.level
    curr_t4_level = naperville.pallet_reserve_t4.level
    curr_t5_level = naperville.pallet_reserve_t5.level

    #Check if Naperville pallet reserve T1 pallets can cover weekly demand
    if naperville.pallet_reserve_t1.level >= pallets_sent:
        #Request Simpy Resource from Naperville
        with naperville.capacity.request() as req:
            yield req

            #Take pallets out of respective turn conatiner
            yield naperville.pallet_reserve_t1.get(pallets_sent)

            #Check if there is pallet need, if so set back to 0
            if naperville.pallet_need.level > 0:
                yield naperville.pallet_need.get(naperville.pallet_need.level)

            #Put pallet need into pallet need container
            yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])


            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallets_sent
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Pallets in transit from Naperville to RDCs
            yield env.timeout(PLANTS_TO_RDC)

        #Run next step in the model, sending pallets from the Naperville to RDCs
        yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 pallet supply
    elif naperville.pallet_reserve_t1.level + naperville.pallet_reserve_t2.level >= pallets_sent:
        #Check if Naperville pallet reserve T2 pallets can cover weekly demand
        if curr_t2_level >= pallets_sent:
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = 0
            pallets_sent_t2 = pallets_sent
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn conatiner
                yield naperville.pallet_reserve_t2.get(pallets_sent_t2)

                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need container
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - naperville.pallet_reserve_t2.level
            
            #Assign pallets sent out of each turn conatiner
            pallets_sent_t1 = pallet_tracker
            pallets_sent_t2 = curr_t2_level
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and naperville.pallet_reserve_t1.level > 0:
                    yield naperville.pallet_reserve_t1.get(pallets_sent_t1)

                yield naperville.pallet_reserve_t2.get(naperville.pallet_reserve_t2.level)

                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need conatiner
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 pallet supply
    elif round(naperville.pallet_reserve_t1.level + naperville.pallet_reserve_t2.level + naperville.pallet_reserve_t3.level, 5) >= pallets_sent:
        #Check if Naperville pallet reserve T3 pallets can cover weekly demand
        if naperville.pallet_reserve_t3.level >= pallets_sent:
            
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
            if abs(naperville.pallet_reserve_t3.level - pallets_sent) < 0.0001:
                pallets_sent_t3 = naperville.pallet_reserve_t3.level
            else:
                pallets_sent_t3 = pallets_sent
            pallets_sent_t4 = 0
            pallets_sent_t5 = 0

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield naperville.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need conatiner
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from the Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
        
        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - naperville.pallet_reserve_t3.level

            #Check to see if weekly demand can be covered only using T2 + T3 pallets
            if pallet_tracker - naperville.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = naperville.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = naperville.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0
            #Must use mix of T1, T2 & T3 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t2.level - naperville.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = naperville.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - naperville.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = naperville.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = naperville.pallet_reserve_t3.level
                pallets_sent_t4 = 0
                pallets_sent_t5 = 0

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and naperville.pallet_reserve_t1.level > 0:
                    yield naperville.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and naperville.pallet_reserve_t2.level > 0:
                    yield naperville.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and naperville.pallet_reserve_t3.level > 0:
                    yield naperville.pallet_reserve_t3.get(pallets_sent_t3)

                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need conatiner
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from the Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 pallet supply
    elif round(naperville.pallet_reserve_t1.level + naperville.pallet_reserve_t2.level + naperville.pallet_reserve_t3.level + naperville.pallet_reserve_t4.level,5) >= pallets_sent:
        #Check if Naperville pallet reserve T4 pallets can cover weekly demand
        if curr_t4_level >= pallets_sent:
            
            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = pallets_sent
            pallets_sent_t5 = 0

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield naperville.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need container
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - naperville.pallet_reserve_t4.level

            #Check to see if weekly demand can be covered only using T3 + T4 pallets
            if pallet_tracker - naperville.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = naperville.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = naperville.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 pallets
            elif pallet_tracker - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = naperville.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - naperville.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = naperville.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = naperville.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Must use mix of T1, T2, T3 & T4 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level - naperville.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = naperville.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = naperville.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = naperville.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = naperville.pallet_reserve_t4.level
                pallets_sent_t5 = 0

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and naperville.pallet_reserve_t1.level > 0:
                    yield naperville.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and naperville.pallet_reserve_t2.level > 0:
                    yield naperville.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and naperville.pallet_reserve_t3.level > 0:
                    yield naperville.pallet_reserve_t3.get(pallets_sent_t3)

                yield naperville.pallet_reserve_t4.get(pallets_sent_t4)

                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need container
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)
            
            #Run next step in the model, sending pallets from Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Check to see if weekly demand can be covered by T1 + T2 + T3 + T4 + T5 pallet supply
    elif round(naperville.pallet_reserve_t1.level + naperville.pallet_reserve_t2.level + naperville.pallet_reserve_t3.level + naperville.pallet_reserve_t4.level + naperville.pallet_reserve_t5.level, 5) >= pallets_sent:
        #Check if Naperville pallet reserve T5 pallets can cover weekly demand
        if curr_t5_level >= pallets_sent:

            #Assign pallets sent out of each turn container
            pallets_sent_t1 = 0
            pallets_sent_t2 = 0
            pallets_sent_t3 = 0
            pallets_sent_t4 = 0
            pallets_sent_t5 = pallets_sent

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn container
                yield naperville.pallet_reserve_t5.get(pallets_sent_t5)

                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need container
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))

        else:
            #Create a pallet tracker bucket for determining how many pallets from each turn container will be used
            pallet_tracker = pallets_sent - naperville.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T4 + T5 pallets
            if pallet_tracker - naperville.pallet_reserve_t4.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                pallets_sent_t3 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t4.level) < 0.01:
                    pallets_sent_t4 = naperville.pallet_reserve_t4.level
                else:
                    pallets_sent_t4 = pallet_tracker
                pallets_sent_t5 = curr_t5_level
            
            #Check to see if weekly demand can be covered only using T3 + T4 + T5 pallets
            elif pallet_tracker - naperville.pallet_reserve_t4.level - naperville.pallet_reserve_t3.level <= 0:
                pallets_sent_t1 = 0
                pallets_sent_t2 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t4.level - naperville.pallet_reserve_t3.level) < 0.01:
                    pallets_sent_t3 = naperville.pallet_reserve_t3.level
                else:
                    pallets_sent_t3 = pallet_tracker - naperville.pallet_reserve_t4.level 
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = naperville.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = naperville.pallet_reserve_t5.level

            #Check to see if weekly demand can be covered only using T2 + T3 + T4 + T5 pallets
            elif pallet_tracker - naperville.pallet_reserve_t4.level - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level <= 0:
                pallets_sent_t1 = 0
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t4.level - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level) < 0.01:
                    pallets_sent_t2 = naperville.pallet_reserve_t2.level
                else:
                    pallets_sent_t2 = pallet_tracker - naperville.pallet_reserve_t4.level - naperville.pallet_reserve_t3.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = naperville.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = naperville.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = naperville.pallet_reserve_t5.level
            
            #Must use a mix of T1, T2, T3, T4 & T5 pallets to cover weekly demand
            else:
                #Conditional check to resolve rounding issue with Simpy to make sure puts/gets do not fail
                if abs(pallet_tracker - naperville.pallet_reserve_t4.level - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level - naperville.pallet_reserve_t1.level) < 0.01:
                    pallets_sent_t1 = naperville.pallet_reserve_t1.level
                else:
                    pallets_sent_t1 = pallet_tracker - naperville.pallet_reserve_t4.level - naperville.pallet_reserve_t3.level - naperville.pallet_reserve_t2.level
                #All (if any) T2 pallets will be sent
                pallets_sent_t2 = naperville.pallet_reserve_t2.level
                #All (if any) T3 pallets will be sent
                pallets_sent_t3 = naperville.pallet_reserve_t3.level
                #All (if any) T4 pallets will be sent
                pallets_sent_t4 = naperville.pallet_reserve_t4.level
                #All (if any) T5 pallets will be sent
                pallets_sent_t5 = naperville.pallet_reserve_t5.level

            #Request Simpy Resource from Naperville
            with naperville.capacity.request() as req:
                yield req

                #Take pallets out of respective turn containers
                if pallets_sent_t1 > 0 and naperville.pallet_reserve_t1.level > 0:
                    yield naperville.pallet_reserve_t1.get(pallets_sent_t1)

                if pallets_sent_t2 > 0 and naperville.pallet_reserve_t2.level > 0:
                    yield naperville.pallet_reserve_t2.get(pallets_sent_t2)

                if pallets_sent_t3 > 0 and naperville.pallet_reserve_t3.level > 0:
                    yield naperville.pallet_reserve_t3.get(pallets_sent_t3)

                if pallets_sent_t4 > 0 and naperville.pallet_reserve_t4.level > 0:
                    yield naperville.pallet_reserve_t4.get(pallets_sent_t4)

                yield naperville.pallet_reserve_t5.get(pallets_sent_t5)


                #Check if there is pallet need, if so set back to 0
                if naperville.pallet_need.level > 0:
                    yield naperville.pallet_need.get(naperville.pallet_need.level)

                #Put pallet need into pallet need container
                yield naperville.pallet_need.put(naperville_rdc_weekly_outbound[week + 2])

                #Pallets in transit from Naperville to RDCs
                yield env.timeout(PLANTS_TO_RDC)

            #Run next step in the model, sending pallets from Naperville to RDCs
            yield env.process(rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))
    
    #Can't fill weekly demand, must buy pallets from supplier
    else:
        #Calculate total weekly demand Naperville can fulfill with current total pallet level
        fillable_demand = naperville.pallet_reserve_t1.level + naperville.pallet_reserve_t2.level + naperville.pallet_reserve_t3.level + naperville.pallet_reserve_t4.level + naperville.pallet_reserve_t5.level
 
        #Request Simpy Resource from Naperville
        with naperville.capacity.request() as req:
            yield req

            #Simulate buying pallets from supplier to cover missing weekly demand
            yield naperville.pallet_reserve_t1.put(pallets_sent - fillable_demand)
            #Take pallet need out as missing pallets have now been sourced
            #yield naperville.pallet_need.get(pallets_sent - fillable_demand)

        
        #Re-run this process, sending pallets from Naperville to RDC
        yield env.process(naperville_to_rdc(env, turns, origin, mexico_hub, richmond, portland, chicago, naperville))

Naperville Return

In [243]:
def dsd_return_naperville(env, naperville):
    '''This process returns pallets back to Naperville'''

    #Locally initiate pallet availability containers
    global pallets_available_to_return_t2
    global pallets_available_to_return_t3
    global pallets_available_to_return_t4
    global pallets_available_to_return_t5

    #Indicate destination for model output
    destination = 'Naperville'

    #Get current avaialable to return pallet level for each turn container
    curr_pallets_t2 = pallets_available_to_return_t2
    curr_pallets_t3 = pallets_available_to_return_t3
    curr_pallets_t4 = pallets_available_to_return_t4
    curr_pallets_t5 = pallets_available_to_return_t5
    

    #Assign current Naperville Hub pallet need for use in formulas
    naperville_need = naperville.pallet_need.level

    #Check if Naperville has any pallet need in current time step
    if naperville_need > 0:

        #Starting in reverse order, check if there is enough turn 5 availability to cover Naperville pallet need
        if curr_pallets_t5 >= naperville_need:
            #If so, take Naperville need pallets out of available to return container
            pallets_available_to_return_t5 -= naperville_need

            #Request Simpy Resource from Naperville
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Naperville
                yield naperville.pallet_reserve_intransit.put(naperville_need)

            #Pallets in transit from DSD back to Naperville
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield naperville.pallet_reserve_intransit.get(naperville_need)
                
                #Put current Naperville Hub pallet need back into Naperville Hub pallet reserve container once returned
                yield naperville.pallet_reserve_t5.put(naperville_need)

                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += naperville_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': naperville_need})

                #Take pallet need out of Naperville Hub once returned
                #yield naperville.pallet_need.get(naperville_need)


        #Check if T4 + T5 pallets can cover Naperville need
        elif curr_pallets_t5 + curr_pallets_t4 >= naperville_need:
            #All (if any) T5 pallets will be returned
            naperville_need_t5 = curr_pallets_t5
            #Calculate T4 pallets that will be sent
            naperville_need_t4 = naperville_need - curr_pallets_t5

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 -= naperville_need_t4


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req
                #Flag pallets are on their way back to Naperville
                yield naperville.pallet_reserve_intransit.put(naperville_need)

            #Pallets in transit from DSD back to Naperville
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield naperville.pallet_reserve_intransit.get(naperville_need)

                #Put current Naperville Hub pallet need back into Naperville Hub pallet reserve container once returned
                if naperville_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Naperville T5 container
                    yield naperville.pallet_reserve_t5.put(naperville_need_t5)

                if naperville_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Naperville T4 container
                    yield naperville.pallet_reserve_t4.put(naperville_need_t4)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += naperville_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': naperville_need})

                #Take pallet need out of Naperville container once pallets are returned
                #yield naperville.pallet_need.get(naperville_need)


        #Check if T3 + T4 + T5 pallets can cover Naperville need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 >= naperville_need:
            #All (if any) T5 pallets will be returned
            naperville_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            naperville_need_t4 = curr_pallets_t4
            #Calculate T3 pallets that will be sent
            naperville_need_t3 = naperville_need - curr_pallets_t5 - curr_pallets_t4

            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 -= naperville_need_t3


            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Naperville
                yield naperville.pallet_reserve_intransit.put(naperville_need)

            #Pallets in transit from DSD back to Naperville
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield naperville.pallet_reserve_intransit.get(naperville_need)

                #Put current Naperville Hub pallet need back into Naperville Hub pallet reserve container once returned
                if naperville_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Naperville T5 container
                    yield naperville.pallet_reserve_t5.put(naperville_need_t5)

                if naperville_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Naperville T4 container
                    yield naperville.pallet_reserve_t4.put(naperville_need_t4)

                if naperville_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Naperville T3 container
                    yield naperville.pallet_reserve_t3.put(naperville_need_t3)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += naperville_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': naperville_need})

                #Take pallet need out of Naperville container once pallets are returned
                #yield naperville.pallet_need.get(naperville_need)


        #Check if T2 + T3 + T4 + T5 pallets can cover Naperville need
        elif curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 >= naperville_need:
            #All (if any) T5 pallets will be returned
            naperville_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            naperville_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            naperville_need_t3 = curr_pallets_t3
            #Calculate T2 pallets that will be sent
            naperville_need_t2 = naperville_need - curr_pallets_t5 - curr_pallets_t4 - curr_pallets_t3
            print('TIME', env.now, 'pallets t2 BEFORE', pallets_available_to_return_t2)
            #Adjust pallets available to return containers
            pallets_available_to_return_t5 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t2 -= naperville_need_t2

            print('TIME Naperville', env.now, 'need', naperville_need)
            print('TIME', env.now, 't2 need', naperville_need_t2)
            print('TIME', env.now, 't3 need', naperville_need_t3)
            print('TIME', env.now, 'pallets t2', pallets_available_to_return_t2)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Flag pallets are on their way back to Naperville
                yield naperville.pallet_reserve_intransit.put(naperville_need)

            #Pallets in transit from DSD back to Naperville Hub
            yield env.timeout(END_TO_PLANT)

            #Request Simpy Resource from PATR
            with patr_bucket.request() as req:
                yield req

                #Take out pallets in transit as they have arrived at the facility
                yield naperville.pallet_reserve_intransit.get(naperville_need)

                #Put current Naperville Hub pallet need back into Naperville Hub pallet reserve container once returned
                if naperville_need_t5 > 0:
                    #Put pallets returned from T5 PATR container back into Naperville T5 container
                    yield naperville.pallet_reserve_t5.put(naperville_need_t5)

                if naperville_need_t4 > 0:
                    #Put pallets returned from T4 PATR container back into Naperville T4 container
                    yield naperville.pallet_reserve_t4.put(naperville_need_t4)

                if naperville_need_t3 > 0:
                    #Put pallets returned from T3 PATR container back into Naperville T3 container
                    yield naperville.pallet_reserve_t3.put(naperville_need_t3)

                if naperville_need_t2 > 0:
                    #Put pallets returned from T2 PATR container back into Naperville T2 container
                    yield naperville.pallet_reserve_t2.put(naperville_need_t2)
                
                #Floor the simulation day to calculate current week's return
                week = env.now//7
                weekly_return_counter[week] += naperville_need

                #Add weekly return by location for dashboard use
                return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': naperville_need})
                
                #Take pallet need out of Naperville container once pallets are returned
                #yield naperville.pallet_need.get(naperville_need)


        #Can't fill full pallet need in Naperville with pallet return program, send everything that is available
        else:
            #All (if any) T5 pallets will be returned
            naperville_need_t5 = curr_pallets_t5
            #All (if any) T4 pallets will be returned
            naperville_need_t4 = curr_pallets_t4
            #All (if any) T3 pallets will be returned
            naperville_need_t3 = curr_pallets_t3
            #All (if any) T2 pallets will be returned
            naperville_need_t2 = curr_pallets_t2

            #Adjust pallets available to return containers
            pallets_available_to_return_t2 = 0
            pallets_available_to_return_t3 = 0
            pallets_available_to_return_t4 = 0
            pallets_available_to_return_t5 = 0

            #Check to make sure there are any pallets available to return
            if curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2 > 0:
                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Flag pallets are on their way back to Naperville
                    yield naperville.pallet_reserve_intransit.put(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                #Pallets in transit from DSD back to Naperville Hub
                yield env.timeout(END_TO_PLANT)

                #Request Simpy Resource from PATR
                with patr_bucket.request() as req:
                    yield req

                    #Take out pallets in transit as they have arrived at the facility
                    yield naperville.pallet_reserve_intransit.get(curr_pallets_t5 + curr_pallets_t4 + curr_pallets_t3 + curr_pallets_t2)

                    #Put current Naperville Hub pallet need back into Naperville Hub pallet reserve container once returned
                    if naperville_need_t5 > 0:
                        #Put pallets returned from T5 PATR container back into Naperville T5 container
                        yield naperville.pallet_reserve_t5.put(naperville_need_t5)

                    if naperville_need_t4 > 0:
                        #Put pallets returned from T4 PATR container back into Naperville T4 container
                        yield naperville.pallet_reserve_t4.put(naperville_need_t4)

                    if naperville_need_t3 > 0:
                        #Put pallets returned from T3 PATR container back into Naperville T3 container
                        yield naperville.pallet_reserve_t3.put(naperville_need_t3)

                    if naperville_need_t2 > 0:
                        #Put pallets returned from T2 PATR container back into Naperville T2 container
                        yield naperville.pallet_reserve_t2.put(naperville_need_t2)
                    
                    #Floor the simulation day to calculate current week's return
                    week = env.now//7
                    weekly_return_counter[week] += (naperville_need_t2 + naperville_need_t3 + naperville_need_t4 + naperville_need_t5)

                    #Add weekly return by location for dashboard use
                    return_by_location.append({'Location': destination, 'week': week + 1, 'Expected Pallet Return': (naperville_need_t2 + naperville_need_t3 + naperville_need_t4 + naperville_need_t5)})

                    #Take pallet need out of Naperville container once pallets are returned
                    #yield naperville.pallet_need.get(naperville_need_t2 + naperville_need_t3 + naperville_need_t4 + naperville_need_t5)


            #No pallets avaialable left to return in the current time step
            else:
                print('No pallets available to return in Naperville')

------

RDC to DSD

In [244]:
def rdc_to_dsd(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets from rdcs to dsd branches'''

    #Bring in global pallet loss tracker from RDC to customers
    global pallets_lost_to_customers

    #Calculate total distribution of pallets that is sent from RDC to customers (all are lost)
    pallets_to_customers = (pallets_sent_t1 + pallets_sent_t2 + pallets_sent_t3 + pallets_sent_t4 + pallets_sent_t5) * RDC_TO_CUSTOMER_DIST
    #Add lost pallets to counter
    pallets_lost_to_customers += pallets_to_customers

    #Calculate pallets sent from RDCs to DSD out of entire pallet pool sent by turn container
    pallets_sent_t1 = pallets_sent_t1 * RDC_TO_DSD_DIST
    pallets_sent_t2 = pallets_sent_t2 * RDC_TO_DSD_DIST
    pallets_sent_t3 = pallets_sent_t3 * RDC_TO_DSD_DIST
    pallets_sent_t4 = pallets_sent_t4 * RDC_TO_DSD_DIST
    pallets_sent_t5 = pallets_sent_t5 * RDC_TO_DSD_DIST

    #Pallets are received at RDCs and held before being sent to other nodes
    yield env.timeout(RDC_HOLDING_TIME)
    
    #Pallets in transit from RDCs to DSD Branches
    yield env.timeout(RDC_TO_DSD)
    
    #Run next step in the model, sending pallets from DSD Branches to DSD Customers
    yield env.process(dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville))


DSD to Customers

In [245]:
def dsd_to_customers_rdc(env, origin, turns, pallets_sent_t1, pallets_sent_t2, pallets_sent_t3, pallets_sent_t4, pallets_sent_t5, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets that came from RDCs to DSD Customers'''

    #Locally initiate returnable pallet containers by turn
    global pallets_available_to_return_t2
    global pallets_available_to_return_t3
    global pallets_available_to_return_t4
    global pallets_available_to_return_t5

    #Locally initiate container that tracks how many pallets have been lost to customers throughout the simulation
    global pallets_lost_to_customers

    #Pallets must wait once they arrive in DSD Branches before they can be sent to customers
    yield env.timeout(DSD_HOLDING_TIME)

    #Pallets in transit from DSD Branches to customers and back to branches
    yield env.timeout(DSD_TO_CUSTOMER)

    print('time', env.now, 'origin', origin)
    print('Calculating pallet loss and damage.....')

    #Calculate pallets remaining after loss and damage on THIS turn
    pallets_remaining_t1 = pallets_sent_t1 * (1 - (PALLET_LOSS_RATE + PALLET_DAMAGE_RATE_T1))
    pallets_remaining_t2 = pallets_sent_t2 * (1 - (PALLET_LOSS_RATE + PALLET_DAMAGE_RATE_T2))
    pallets_remaining_t3 = pallets_sent_t3 * (1 - (PALLET_LOSS_RATE + PALLET_DAMAGE_RATE_T3))
    pallets_remaining_t4 = pallets_sent_t4 * (1 - (PALLET_LOSS_RATE + PALLET_DAMAGE_RATE_T4))
    pallets_remaining_t5 = 0

    #Calculate pallets lost to customers based on pallet loss rate by turn
    pallets_lost_t1 = pallets_sent_t1 * PALLET_LOSS_RATE
    pallets_lost_t2 = pallets_sent_t2 * PALLET_LOSS_RATE
    pallets_lost_t3 = pallets_sent_t3 * PALLET_LOSS_RATE
    pallets_lost_t4 = pallets_sent_t4 * PALLET_LOSS_RATE
    pallets_lost_t5 = pallets_sent_t5 * PALLET_LOSS_RATE

    #Calculate total pallets that are sent to customers from DSD branches accounting for loss and damage rates
    total_pallets_sent = pallets_sent_t1 + pallets_sent_t2 + pallets_sent_t3 + pallets_sent_t4 + pallets_sent_t5

    #In this simulation - We buy pallets in US plants until their need can be filled by mexico recycled pallets
    #Mexico will ALWAYS buy pallets

    #Check origin of incoming pallets

    if origin == 'mexico':
        #Check turn
        if turns == 3:
                #Add pallets that have been recycled from turn 1 to the turn 2 container
                pallets_available_to_return_t2 += pallets_remaining_t1
                print('time', env.now, '-', origin, f'Pallets sent to t2: {pallets_remaining_t1}')
                #Logic check to correctly reset pallets available return container to 0 due to some timing issues that arise
                if pallets_available_to_return_t3 - pallets_remaining_t3 < 0:
                    pallets_available_to_return_t3 = 0

                #No timing issues
                else:
                    #Subtract pallets that have "died" due to going through their last turn
                    pallets_available_to_return_t3 -= pallets_remaining_t3
                #Add pallets that have been recycled from turn 2 to the turn 3 container
                pallets_available_to_return_t3 += pallets_remaining_t2
                print('time', env.now, '-', origin, f'Pallets sent to t3: {pallets_remaining_t2}')
        #Check turn
        elif turns == 4:
            #Add pallets that have been recycled from turn 1 to the turn 2 container
            pallets_available_to_return_t2 += pallets_remaining_t1
            #Add pallets that have been recycled from turn 2 to the turn 3 container
            pallets_available_to_return_t3 += pallets_remaining_t2
            #Logic check to correctly reset pallets available return container to 0 due to some timing issues that arise
            if pallets_available_to_return_t4 - pallets_remaining_t4 < 0:
                pallets_available_to_return_t4 = 0

            #No timing issues
            else:
                #Subtract pallets that have "died" due to going through their last turn
                pallets_available_to_return_t4 -= pallets_remaining_t4
            #Add pallets that have been recycled from turn 3 to the turn 4 container
            pallets_available_to_return_t4 += pallets_remaining_t3
        
        #Must be 5th turn
        else:
            #Add pallets that have been recycled from turn 1 to the turn 2 container
            pallets_available_to_return_t2 += pallets_remaining_t1
            #Add pallets that have been recycled from turn 2 to the turn 3 container
            pallets_available_to_return_t3 += pallets_remaining_t2
            #Add pallets that have been recycled from turn 3 to the turn 4 container
            pallets_available_to_return_t4 += pallets_remaining_t3
            #Add pallets that have been recycled from turn 4 to the turn 5 container
            pallets_available_to_return_t5 += pallets_remaining_t4
    
    else:
        #Logic check to see if there are any non turn 1 pallets in this batch
        if total_pallets_sent - pallets_sent_t1 > 0:
            #Check turn
            if turns == 3:
                #Logic check to correctly reset pallets available return container to 0 due to some timing issues that arise
                if pallets_available_to_return_t3 - pallets_remaining_t3 < 0:
                    pallets_available_to_return_t3 = 0
                #No timing issues
                else:
                    #Subtract pallets that have "died" due to going through their last turn
                    pallets_available_to_return_t3 -= pallets_remaining_t3
                #Add pallets that have been recycled from turn 2 to the turn 3 container
                pallets_available_to_return_t3 += pallets_remaining_t2
                print('time', env.now, '-', origin, f'Pallets sent to t3: {pallets_remaining_t2}')

            #Check turn
            elif turns == 4:
                #Add pallets that have been recycled from turn 2 to the turn 3 container
                pallets_available_to_return_t3 += pallets_remaining_t2
                #Logic check to correctly reset pallets available return container to 0 due to some timing issues that arise
                if pallets_available_to_return_t4 - pallets_remaining_t4 < 0:
                    pallets_available_to_return_t4 = 0
                #No timing issues
                else:
                    #Subtract pallets that have "died" due to going through their last turn
                    pallets_available_to_return_t4 -= pallets_remaining_t4
                #Add pallets that have been recycled from turn 3 to the turn 4 container
                pallets_available_to_return_t4 += pallets_remaining_t3

            #Must be 5th turn
            else:
                #Add pallets that have been recycled from turn 2 to the turn 3 container
                pallets_available_to_return_t3 += pallets_remaining_t2
                #Add pallets that have been recycled from turn 3 to the turn 4 container
                pallets_available_to_return_t4 += pallets_remaining_t3
                #Add pallets that have been recycled from turn 4 to the turn 5 container
                pallets_available_to_return_t5 += pallets_remaining_t4

------

# Simulation Set Up

In [246]:
#Set random seed for reproducibility
RANDOM_SEED = 9
random.seed(RANDOM_SEED)

#Instantiate simpy environment for simulation to run in
env = simpy.Environment()

#Create PATR resource for use in simulation
patr_bucket = simpy.Resource(env, capacity = 1000)

#Instantiate Mexico class for simualation
mexico_hub = Mexico_Hub(env, initial_pallets = mexico_hub_dsd_weekly_outbound[0] + mexico_hub_rdc_weekly_outbound[0])
#Instantiate Richmond class for simualation
richmond = Richmond(env, initial_pallets = richmond_dsd_weekly_outbound[0] + richmond_rdc_weekly_outbound[0])
#Instantiate Portland class for simualation
portland = Portland(env, initial_pallets = portland_dsd_weekly_outbound[0] + portland_rdc_weekly_outbound[0])
#Instantiate Chicago class for simualation
chicago = Chicago(env, initial_pallets = chicago_dsd_weekly_outbound[0] + chicago_rdc_weekly_outbound[0])
#Instantiate Napoerville class for simualation
naperville = Naperville(env, initial_pallets = naperville_rdc_weekly_outbound[0])

#Set number of turns pallets can be recycled through the network
turns = 3

Functions for simulation run

In [247]:
#Create return function
def return_pals(env, richmond, portland, chicago, naperville):
    '''This process runs the pallet return program and sends pallets back to each location'''
    env.process(dsd_return_richmond(env, richmond))
    yield env.timeout(0)
    env.process(dsd_return_portland(env, portland))
    yield env.timeout(0)
    env.process(dsd_return_chicago(env, chicago))
    yield env.timeout(0)
    env.process(dsd_return_naperville(env, naperville))
    #yield env.timeout(0)
    #env.process(dsd_return_salinas(env, mexico_hub))

In [248]:
#Create functions to send pallets out of each location through the network
def send_pallets_richmond(env, turns, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets out of Richmond to both RDCs & direct to DSD branches'''
    env.process(richmond_to_rdc(env, turns, 'richmond', mexico_hub, richmond, portland, chicago, naperville))    
    yield env.timeout(1)
    env.process(richmond_to_dsd(env, turns, 'richmond', mexico_hub, richmond, portland, chicago, naperville))

def send_pallets_mexico(env, turns, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets out of Mexico to RDCs, direct to DSD branches & direct to customers'''
    env.process(salinas_to_rdc(env, turns, 'mexico', mexico_hub, richmond, portland, chicago, naperville))
    yield env.timeout(1)
    env.process(salinas_to_dsd(env, turns, 'mexico', mexico_hub, richmond, portland, chicago, naperville))

def send_pallets_portland(env, turns, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets out of Portland to both RDCs & direct to DSD branches'''
    env.process(portland_to_rdc(env, turns, 'portland', mexico_hub, richmond, portland, chicago, naperville))
    yield env.timeout(1)
    env.process(portland_to_dsd(env, turns, 'portland', mexico_hub, richmond, portland, chicago, naperville))

def send_pallets_chicago(env, turns, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends pallets out of Chicago to both RDCs & direct to DSD branches'''
    env.process(chicago_to_rdc(env, turns, 'chicago', mexico_hub, richmond, portland, chicago, naperville))
    yield env.timeout(1)
    env.process(chicago_to_dsd(env, turns, 'chicago', mexico_hub, richmond, portland, chicago, naperville))

In [249]:
def send_weekly_pallets(env, turns, mexico_hub, richmond, portland, chicago, naperville):
    '''This process sends out the initial pallets from each location'''
    #Mexico
    env.process(send_pallets_mexico(env, turns, mexico_hub, richmond, portland, chicago, naperville))

    #US Plants
    env.process(send_pallets_richmond(env, turns, mexico_hub, richmond, portland, chicago, naperville))
    env.process(send_pallets_portland(env, turns, mexico_hub, richmond, portland, chicago, naperville))
    env.process(send_pallets_chicago(env, turns, mexico_hub, richmond, portland, chicago, naperville))
    env.process(naperville_to_rdc(env, turns, 'naperville', mexico_hub, richmond, portland, chicago, naperville))


def send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, delay):
    "This process sends out pallets from each location with a predetermined delay of time"
    #Mexico
    delivery = start_delayed(env, send_pallets_mexico(env, turns, mexico_hub, richmond, portland, chicago, naperville), delay)

    #US plants
    delivery = start_delayed(env, send_pallets_richmond(env, turns, mexico_hub, richmond, portland, chicago, naperville), delay)
    delivery = start_delayed(env, send_pallets_portland(env, turns, mexico_hub, richmond, portland, chicago, naperville), delay)
    delivery = start_delayed(env, send_pallets_chicago(env, turns, mexico_hub, richmond, portland, chicago, naperville), delay)
    delivery = start_delayed(env, naperville_to_rdc(env, turns, 'naperville', mexico_hub, richmond, portland, chicago, naperville), delay)

----

### Run Simulation

Outbound

In [250]:
#Send out first week of pallets
#send_weekly_pallets(env, turns, mexico_hub, richmond, portland, chicago, naperville)

#Year 1
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 7)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 14)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 21)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 28)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 35)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 42)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 49)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 56)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 63)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 70)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 77)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 84)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 91)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 98)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 105)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 112)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 119)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 126)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 133)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 140)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 147)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 154)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 161)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 168)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 175)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 182)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 189)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 196)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 203)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 210)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 217)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 224)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 231)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 238)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 245)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 252)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 259)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 266)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 273)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 280)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 287)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 294)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 301)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 308)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 315)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 322)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 329)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 336)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 343)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 350)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 357)
send_weekly_pallets_delay(env, turns, mexico_hub, richmond, portland, chicago, naperville, 364)


Return

In [251]:
#Year 1
#start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 25)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 32)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 39)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 46)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 53)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 60)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 67)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 74)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 81)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 88)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 95)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 102)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 109)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 116)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 123)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 130)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 137)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 144)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 151)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 158)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 165)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 172)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 179)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 186)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 193)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 200)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 207)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 214)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 221)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 228)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 235)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 242)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 249)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 256)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 263)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 270)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 277)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 284)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 291)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 298)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 305)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 312)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 319)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 326)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 333)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 340)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 347)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 354)
start_delayed(env, return_pals(env, richmond, portland, chicago, naperville), 361)

<Process(starter) object at 0x2d24f9b5bd0>

Run

In [252]:
#Run simulation for one year
env.run(until = 370)

time 7 salinas_to_rdc running
time 7 pallets_sent 4410
time 7 richmond_to_rdc running
time 7 portland_to_rdc running
time 7 chicago_to_rdc running
time 7 naperville_to_rdc running
time 8 salinas_to_dsd running
time 8 pallets_sent 5310
time 8 richmond_to_dsd running
week 1
time 8 portland_to_dsd running
time 8 chicago-to_dsd running
time 14 salinas_to_rdc running
time 14 pallets_sent 7380
time 14 richmond_to_rdc running
time 14 portland_to_rdc running
time 14 chicago_to_rdc running
time 14 naperville_to_rdc running
time 14 salinas_to_rdc running
time 14 pallets_sent 7380
time 14 richmond_to_rdc running
time 14 portland_to_rdc running
time 14 chicago_to_rdc running
time 14 naperville_to_rdc running
time 15 salinas_to_dsd running
time 15 pallets_sent 5820
time 15 richmond_to_dsd running
week 2
time 15 portland_to_dsd running
time 15 chicago-to_dsd running
time 15 salinas_to_dsd running
time 15 pallets_sent 5820
time 15 richmond_to_dsd running
week 2
time 15 portland_to_dsd running
time 15

-----

#### Output

In [253]:
def week_to_str(week):
    '''transform week from int to "00" string'''
    if len(str(week)) == 1:
        week = '0' + str(week)
    else:
        week = str(week)
    return week

In [254]:
#transform pallet surplus weekly data to dataframe for joining
pallet_surplus = pd.DataFrame(output)

In [255]:
#transform pallet return weekly data to dataframe for joining
pallet_return = pd.DataFrame.from_dict(dict(enumerate(weekly_return_counter, start = 1)), orient = 'index', columns = ['expected_pallet_return']).reset_index()
pallet_return = pallet_return.rename(columns={'index': 'week'})

#Tranform week to string "00" format for dashboard
pallet_return['week'] = pallet_return['week'].apply(week_to_str)

In [256]:
#merge pallet return & pallet surplus weekly data for output
weekly_pallet_data = pallet_return.merge(pallet_surplus, on = 'week')

In [257]:
weekly_pallet_data

,week,expected_pallet_return,day,t2,t3,expected_pallet_surplus
0,01,0.0,0,0.000000,0.000000,0.000000
1,01,0.0,1,0.000000,0.000000,0.000000
2,01,0.0,2,0.000000,0.000000,0.000000
3,01,0.0,3,0.000000,0.000000,0.000000
4,01,0.0,4,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
358,53,9850.0,365,73140.388242,0.000000,73140.388242
359,53,9850.0,366,76785.388242,0.000000,76785.388242
360,53,9850.0,367,76785.388242,1413.241275,78198.629517
361,53,9850.0,368,82023.388242,1413.241275,83436.629517


In [258]:
#Tie up all the weekly demand for output purposes

#w/o Salinas
weekly_demand = [sum(x) for x in zip(richmond_dsd_weekly_outbound, richmond_rdc_weekly_outbound, chicago_dsd_weekly_outbound, chicago_rdc_weekly_outbound,
    portland_dsd_weekly_outbound, portland_rdc_weekly_outbound, naperville_rdc_weekly_outbound)]

#With Salinas
#weekly_demand = [sum(x) for x in zip(mexico_hub_dsd_weekly_outbound, mexico_hub_rdc_weekly_outbound, richmond_dsd_weekly_outbound, richmond_rdc_weekly_outbound, chicago_dsd_weekly_outbound, chicago_rdc_weekly_outbound,
    #portland_dsd_weekly_outbound, portland_rdc_weekly_outbound, naperville_rdc_weekly_outbound)]

In [259]:
#Repurpose weekly demand to join on final dataframe
weekly_demand_df = pd.DataFrame.from_dict(dict(enumerate(weekly_demand, start = 1)), orient = 'index', columns = ['weekly_demand']).reset_index()

#Rename index column for joining purposes
weekly_demand_df = weekly_demand_df.rename({'index': 'week'}, axis = 1)

#Transform week to "00" format for dashboard
weekly_demand_df['week'] = weekly_demand_df['week'].apply(week_to_str)

In [260]:
#Add weekly demand to final dataframe
weekly_pallet_data =weekly_pallet_data.merge(weekly_demand_df, on = 'week')

In [261]:
#reorder columns for output
weekly_pallet_data = weekly_pallet_data.iloc[:, [2,0,6,1,3,4,5]]

In [262]:
#Add year for dashboard purposes
weekly_pallet_data['year'] = '2024'

In [263]:
weekly_pallet_data.head()

,day,week,weekly_demand,expected_pallet_return,t2,t3,expected_pallet_surplus,year
0,0,01,8335,0.0,0.0,0.0,0.0,2024
1,1,01,8335,0.0,0.0,0.0,0.0,2024
2,2,01,8335,0.0,0.0,0.0,0.0,2024
3,3,01,8335,0.0,0.0,0.0,0.0,2024
4,4,01,8335,0.0,0.0,0.0,0.0,2024


In [264]:
#No Salinas
weekly_pallet_data.to_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\Simulation\turns_data.csv")


#Salinas
#weekly_pallet_data.to_csv(r"C:\Users\BMZ7571\OneDrive - MDLZ\Desktop\turns_data_w_salinas.csv")

In [265]:
#Create dataframe to track expected returns by each location
return_by_location_df = pd.DataFrame(return_by_location)

In [266]:
#Return by location output
return_by_location_df.to_csv(r"\\mznapwapalt002\alteryx\NA_BIS\Everything On Pallets\Simulation\returns_by_location.csv")